In [1]:
# FILE NAME: Collecting data hh.ru
# PROGRAMMER: VG6
# DATE: 26.03.2020
# Purpose: To collect valuable information from hh ru
!python -m pip install matplotlib

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [15]:
# importing modules
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import re
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
# initializing path to firefox driver
firefox_driver = 'C:\\Users\Gololobov\Documents\GitHub\Collecting_data_openedu\geckodriver.exe'

# initializing firefox options instance 
options = webdriver.FirefoxOptions()

# initializing headless options
options.add_argument('headless')

# initializing browser with driver and options
browser = webdriver.Firefox(executable_path=firefox_driver, options=options)


In [17]:
# initializing html tags patterns
tags = re.compile(r'<[^>]+>')

# Removing html tags from text function
# Arguments: 
#   - text we want to remove tags from
#      text : str
# Returns:
#   - string with removed html texts
#      (str)
def remove_html_tags(text : str) -> str:
    return tags.sub(' ', text)

In [18]:
# Getting course information function
# Arguments:
#   - Link to course
#       link_to_the_course : str,
# Returns:
#   - DataFrame with full information about the course
#       (pd.DataFrame)
def get_course_info(link_to_the_course : str) -> pd.DataFrame:
    browser.get(link_to_the_course)

    # initializing beautiful soup class instance to parse current page
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    # getting vacancy title
    vacancy_title = soup.find('h1', {'data-qa' : 'vacancy-title'})

    if vacancy_title != None:
        vacancy_title = vacancy_title.text
    else:
        vacancy_title = " "

    # getting salary
    vacancy_salary = soup.find('p', {'class' : 'vacancy-salary'})

    if vacancy_salary != None:
        vacancy_salary = vacancy_salary.text
    else:
        vacancy_salary = " "

    vacancy_company = soup.find('a', {'data-qa' : 'vacancy-company-name'})

    if vacancy_company != None:
        vacancy_company = vacancy_company.text
    else:
        vacancy_company = " "

    vacancy_location = soup.find('p', {'data-qa' : 'vacancy-view-location'})

    if vacancy_location != None:
        vacancy_location = vacancy_location.text
    else:
        vacancy_location = " "

    vacancy_description = soup.find('div', {'class' : 'vacancy-description'})

    vacancy_experiance = soup.find('span', {'data-qa' : 'vacancy-experience'})
    if vacancy_experiance != None:
        vacancy_experiance = vacancy_experiance.text
    else:
        vacancy_experiance = " "

    vacancy_work_time = soup.find('p', {'data-qa' : 'vacancy-view-employment-mode'})
    if vacancy_work_time != None:
        vacancy_work_time = vacancy_work_time.text
    else:
        vacancy_work_time = " "

    vacancy_key_concepts = soup.find_all('div', {'data-qa' : 'bloko-tag bloko-tag_inline skills-element'})


    if vacancy_key_concepts != []:
        key_conc = []
        for concept in vacancy_key_concepts:
            key_conc.append(concept.text)

        vacancy_key_concepts = ', '.join(key_conc)
    else:
        vacancy_key_concepts = " "

    vacancy_description_names = soup.find_all('strong')

    if vacancy_description_names != []:
        vacancy_desc = []
        for desc in vacancy_description_names:
            vacancy_desc.append(desc.text)

        vacancy_description_names = vacancy_desc

    else:
        vacancy_description_names = " "

    if vacancy_description != None:
        vacancy_description_data = vacancy_description.find_all('ul')
    else:
        vacancy_description_data = []

    if vacancy_description_data != []:
        vacancy_desc = []
        for desc in vacancy_description_data:
            vacancy_desc.append(desc.text)

        vacancy_description_data = vacancy_desc

    else:
        vacancy_description_data = " "

    vacancy_description_names = [x.lower() for x in vacancy_description_names]

    for (name, data) in zip(vacancy_description_names, vacancy_description_data):
        if re.search('обязанности', name):
            print(name)
            vacancy_responsibilities = data
            break
        else:
            vacancy_responsibilities = ' '


    for (name, data) in zip(vacancy_description_names, vacancy_description_data):
        if re.search('требования', name):
            vacancy_requerments = data
            break
        else:
            vacancy_requerments = ''

    vacancy_data = {'Название' : vacancy_title, 'Компания' : vacancy_company, 
                                'Местоположение' : vacancy_location, 'Занятость' : vacancy_work_time,
                                'Требования по умениям' : vacancy_requerments, 'Требования по опыту' : vacancy_experiance,
                                'Должностные обязанности' : vacancy_responsibilities, 'Зарплата' : vacancy_salary,
                   'Ключевые навыки' : vacancy_key_concepts}

    vacancy_data = pd.DataFrame([vacancy_data])
    
    return vacancy_data

In [19]:
get_course_info('https://spb.hh.ru/vacancy/35357861')

обязанности:


,Название,Компания,Местоположение,Занятость,Требования по умениям,Требования по опыту,Должностные обязанности,Зарплата,Ключевые навыки
0,Менеджер по интернет-рекламе,"ТЕРМЕКС, Группа Компаний","Бухарестская, Электросила, Санкт-Петербург, Бл...","Полная занятость, полный день",работа в крупной международной корпорации с м...,1–3 года,"опыт SEO, Yandex Direct, Google Ads; компьюте...",от 48 000 до 60 000 руб. на руки,"SEO оптимизация, Ведение групп в социальных се..."


In [20]:
def get_vacancies_on_page(link):
    
    browser.get(link)

    # initializing beautiful soup class instance to parse current page
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    list_of_dfs = []
    
    links_to_courses = soup.find_all('a', {'data-qa' : 'vacancy-serp__vacancy-title'})

    if links_to_courses != []:
        for link in links_to_courses:
            list_of_dfs.append(get_course_info(link['href']))
            print(link['href'])
            
    # initializing Data Frame with information of all
    # courses on the current page
    page_courses_info = pd.DataFrame()

    # concating all DataFrames we get on previous step
    for data_frame in list_of_dfs:
        # concatinating all courses info dataframes
        page_courses_info = pd.concat([page_courses_info, data_frame], axis=0, ignore_index=True)
        
    return page_courses_info
    
    

In [21]:
def get_vacancies(pages_number, link='https://spb.hh.ru/catalog/Informacionnye-tehnologii-Internet-Telekom'):
    
    list_of_dfs = []
    
    for page in range(pages_number):
        page_link = link + '?page=' + str(page)
        
        page_vac = get_vacancies_on_page(page_link)
        list_of_dfs.append(page_vac)
    
    # initializing Data Frame with information of all
    # courses on the current page
    page_courses_info = pd.DataFrame()

    # concating all DataFrames we get on previous step
    for data_frame in list_of_dfs:
        # concatinating all courses info dataframes
        page_courses_info = pd.concat([page_courses_info, data_frame], axis=0, ignore_index=True)
        
    return page_courses_info
    
        

In [24]:
data = get_vacancies(40)
data.to_csv(r'Z:\Collecting_data_openedu\first.csv')

https://hhcdn.ru/click?b=185506&place=35
обязанности:
https://spb.hh.ru/vacancy/36271867
https://spb.hh.ru/vacancy/31917545
https://spb.hh.ru/vacancy/36523345
https://spb.hh.ru/vacancy/35276136
https://spb.hh.ru/vacancy/36259952
https://spb.hh.ru/vacancy/36461114
https://spb.hh.ru/vacancy/36528350
обязанности: 
https://spb.hh.ru/vacancy/36271679
https://spb.hh.ru/vacancy/36458917
обязанности:
https://spb.hh.ru/vacancy/36032761
https://spb.hh.ru/vacancy/36465065
https://spb.hh.ru/vacancy/36446112
обязанности:
https://spb.hh.ru/vacancy/36217443
https://spb.hh.ru/vacancy/34577987
https://spb.hh.ru/vacancy/36442924
https://spb.hh.ru/vacancy/36191145
обязанности:
https://spb.hh.ru/vacancy/36228963
обязанности:
https://spb.hh.ru/vacancy/36378385
https://spb.hh.ru/vacancy/36293071
обязанности:
https://spb.hh.ru/vacancy/36291789
https://spb.hh.ru/vacancy/36291094
обязанности:
https://spb.hh.ru/vacancy/36144646
https://spb.hh.ru/vacancy/36283835
https://spb.hh.ru/vacancy/35076726
обязанности:
h

https://spb.hh.ru/vacancy/36146851
обязанности: 
https://spb.hh.ru/vacancy/35229599
https://spb.hh.ru/vacancy/36370795
https://spb.hh.ru/vacancy/36438828
обязанности:
https://spb.hh.ru/vacancy/36183959
https://spb.hh.ru/vacancy/36320523
обязанности:
https://spb.hh.ru/vacancy/36215967
обязанности:
https://spb.hh.ru/vacancy/36445686
https://spb.hh.ru/vacancy/36101243
https://spb.hh.ru/vacancy/36036276
обязанности:
https://spb.hh.ru/vacancy/36307814
https://spb.hh.ru/vacancy/36201350
https://spb.hh.ru/vacancy/36183111
обязанности:　
https://spb.hh.ru/vacancy/36136259
https://spb.hh.ru/vacancy/36448760
https://spb.hh.ru/vacancy/36182291
https://spb.hh.ru/vacancy/36158977
https://spb.hh.ru/vacancy/36320051
https://spb.hh.ru/vacancy/36442590
https://spb.hh.ru/vacancy/35032442
обязанности:
https://spb.hh.ru/vacancy/36366232
обязанности:
https://spb.hh.ru/vacancy/36130191
https://spb.hh.ru/vacancy/35599423
основные обязанности позиции:
https://spb.hh.ru/vacancy/36478322
профессиональные обязанн

обязанности:
https://spb.hh.ru/vacancy/36354204
https://spb.hh.ru/vacancy/36428132
обязанности:
https://spb.hh.ru/vacancy/36524306
обязанности:
https://spb.hh.ru/vacancy/36369031
https://spb.hh.ru/vacancy/36254450
https://spb.hh.ru/vacancy/33816104
https://spb.hh.ru/vacancy/35854883
https://spb.hh.ru/vacancy/32654683
https://spb.hh.ru/vacancy/36523976
https://spb.hh.ru/vacancy/36449449
обязанности:
https://spb.hh.ru/vacancy/36437987
обязанности:
https://spb.hh.ru/vacancy/35924291
https://spb.hh.ru/vacancy/36278945
https://spb.hh.ru/vacancy/36142066
https://spb.hh.ru/vacancy/36308542
обязанности:
https://spb.hh.ru/vacancy/36304364
обязанности:
https://spb.hh.ru/vacancy/36526612
обязанности:
https://spb.hh.ru/vacancy/34848161
https://spb.hh.ru/vacancy/36432463
https://spb.hh.ru/vacancy/33512589
https://spb.hh.ru/vacancy/36319675
https://spb.hh.ru/vacancy/36335112
обязанности:
https://spb.hh.ru/vacancy/36141882
https://spb.hh.ru/vacancy/35747614
обязанности:
https://spb.hh.ru/vacancy/3646

обязанности:
https://spb.hh.ru/vacancy/36278482
обязанности:
https://spb.hh.ru/vacancy/36263285
обязанности:
https://spb.hh.ru/vacancy/36225354
https://spb.hh.ru/vacancy/36190670
дистанционный менеджер по продажам ( договор гпх)обязанности:
https://spb.hh.ru/vacancy/36211662
обязанности:
https://spb.hh.ru/vacancy/35559531
обязанности работника:
https://spb.hh.ru/vacancy/36135072
обязанности:
https://spb.hh.ru/vacancy/35324765
обязанности:
https://spb.hh.ru/vacancy/36282397
https://spb.hh.ru/vacancy/36501656
обязанности:
https://spb.hh.ru/vacancy/36480058
https://spb.hh.ru/vacancy/36281981
https://spb.hh.ru/vacancy/36464612
https://spb.hh.ru/vacancy/33600419
обязанности:
https://spb.hh.ru/vacancy/36315807
https://spb.hh.ru/vacancy/36438841
https://spb.hh.ru/vacancy/35447814
https://spb.hh.ru/vacancy/35769342
обязанности:
https://spb.hh.ru/vacancy/36520226
обязанности:
https://spb.hh.ru/vacancy/34947982
обязанности
https://spb.hh.ru/vacancy/36268623
https://spb.hh.ru/vacancy/34685428
htt

обязанности:
https://spb.hh.ru/vacancy/36119316
обязанности:
https://spb.hh.ru/vacancy/36174471
обязанности:
https://spb.hh.ru/vacancy/36394442
https://spb.hh.ru/vacancy/36267049
https://spb.hh.ru/vacancy/36493366
https://spb.hh.ru/vacancy/36058204
https://spb.hh.ru/vacancy/36184439
обязанности:
https://spb.hh.ru/vacancy/36423541
что будет входить в ваши обязанности:
https://spb.hh.ru/vacancy/36320796
https://spb.hh.ru/vacancy/36121630
https://spb.hh.ru/vacancy/36263576
https://spb.hh.ru/vacancy/36124804
https://spb.hh.ru/vacancy/36360004
https://spb.hh.ru/vacancy/36519934
https://spb.hh.ru/vacancy/36389631
обязанности:
https://spb.hh.ru/vacancy/35834576
https://spb.hh.ru/vacancy/36269914
обязанности:
https://spb.hh.ru/vacancy/36135053
https://spb.hh.ru/vacancy/36252371
обязанности:
https://spb.hh.ru/vacancy/35726352
https://spb.hh.ru/vacancy/36426507
https://spb.hh.ru/vacancy/35794682
обязанности:
https://spb.hh.ru/vacancy/34246636
обязанности:
https://spb.hh.ru/vacancy/36359208
обяза

обязанности:
https://spb.hh.ru/vacancy/36243260
https://spb.hh.ru/vacancy/36514013
https://spb.hh.ru/vacancy/36511860
https://spb.hh.ru/vacancy/35605913
обязанности:
https://spb.hh.ru/vacancy/36184318
https://spb.hh.ru/vacancy/35310815
https://spb.hh.ru/vacancy/35965540
обязанности:
https://spb.hh.ru/vacancy/36245245
обязанности:
https://spb.hh.ru/vacancy/35367454
обязанности:
https://spb.hh.ru/vacancy/35430671
https://spb.hh.ru/vacancy/36183877
https://spb.hh.ru/vacancy/36069227
https://spb.hh.ru/vacancy/36237328
обязанности:
https://spb.hh.ru/vacancy/36454446
https://spb.hh.ru/vacancy/36340800
обязанности:
https://spb.hh.ru/vacancy/36310455
https://spb.hh.ru/vacancy/36190693
обязанности:
https://spb.hh.ru/vacancy/34265823
https://spb.hh.ru/vacancy/34538572
https://spb.hh.ru/vacancy/36513386
https://spb.hh.ru/vacancy/36265794
обязанности:
https://spb.hh.ru/vacancy/34974192
https://spb.hh.ru/vacancy/36121928
https://spb.hh.ru/vacancy/36314559
обязанности:
https://spb.hh.ru/vacancy/3616

https://spb.hh.ru/vacancy/36466532
обязанности
https://spb.hh.ru/vacancy/36187466
основные обязанности:
https://spb.hh.ru/vacancy/36499896
обязанности:
https://spb.hh.ru/vacancy/36263291
https://spb.hh.ru/vacancy/36072506
https://spb.hh.ru/vacancy/36390665
https://spb.hh.ru/vacancy/36502153
https://spb.hh.ru/vacancy/35092264
https://spb.hh.ru/vacancy/36499531
https://spb.hh.ru/vacancy/26357017
https://spb.hh.ru/vacancy/36100325
обязанности:
https://spb.hh.ru/vacancy/36154135
https://spb.hh.ru/vacancy/35771872
https://spb.hh.ru/vacancy/36178097
обязанности:
https://spb.hh.ru/vacancy/36501518
https://spb.hh.ru/vacancy/36354143
обязанности:
https://spb.hh.ru/vacancy/34552675
https://spb.hh.ru/vacancy/36112219
https://spb.hh.ru/vacancy/36338084
https://spb.hh.ru/vacancy/36457615
https://spb.hh.ru/vacancy/35816252
https://spb.hh.ru/vacancy/36245672
обязанности:
https://spb.hh.ru/vacancy/36178035
https://spb.hh.ru/vacancy/34015716
https://spb.hh.ru/vacancy/36496045
https://spb.hh.ru/vacancy/

https://spb.hh.ru/vacancy/36167072
обязанности:
https://spb.hh.ru/vacancy/36495095
https://spb.hh.ru/vacancy/31207223
https://spb.hh.ru/vacancy/36108894
обязанности:
https://spb.hh.ru/vacancy/36349084
https://spb.hh.ru/vacancy/35626640
https://spb.hh.ru/vacancy/36235919
обязанности:
https://spb.hh.ru/vacancy/36500775
https://spb.hh.ru/vacancy/36359931
обязанности:
https://spb.hh.ru/vacancy/36173971
https://spb.hh.ru/vacancy/35771872
https://spb.hh.ru/vacancy/36266976
https://spb.hh.ru/vacancy/36068429
https://spb.hh.ru/vacancy/36135369
обязанности:
https://spb.hh.ru/vacancy/36122227
https://spb.hh.ru/vacancy/36505642
https://spb.hh.ru/vacancy/35354564
https://spb.hh.ru/vacancy/34539773
https://spb.hh.ru/vacancy/36338411
https://spb.hh.ru/vacancy/36283179
https://spb.hh.ru/vacancy/36502139
https://spb.hh.ru/vacancy/33052735
https://spb.hh.ru/vacancy/36160021
https://spb.hh.ru/vacancy/32501685
https://spb.hh.ru/vacancy/36065859
https://spb.hh.ru/vacancy/36409793
https://spb.hh.ru/vacancy

https://spb.hh.ru/vacancy/36496418
обязанности:
https://spb.hh.ru/vacancy/36412726
обязанности:
https://spb.hh.ru/vacancy/35467013
https://spb.hh.ru/vacancy/36159994
https://spb.hh.ru/vacancy/35939413
обязанности:
https://spb.hh.ru/vacancy/36091186
https://spb.hh.ru/vacancy/34347403
должностные обязанности:
https://spb.hh.ru/vacancy/36068344
обязанности:
https://spb.hh.ru/vacancy/36097308
обязанности:
https://spb.hh.ru/vacancy/36384890
https://spb.hh.ru/vacancy/36458779
https://spb.hh.ru/vacancy/36080333
обязанности:
https://spb.hh.ru/vacancy/36393837
https://spb.hh.ru/vacancy/35988758
https://spb.hh.ru/vacancy/36160488
https://spb.hh.ru/vacancy/36229829
https://spb.hh.ru/vacancy/36157508
https://spb.hh.ru/vacancy/34891002
обязанности:
https://spb.hh.ru/vacancy/36338037
https://spb.hh.ru/vacancy/36235472
https://spb.hh.ru/vacancy/36386033
https://spb.hh.ru/vacancy/35302034
https://spb.hh.ru/vacancy/36336177
https://spb.hh.ru/vacancy/34241349
обязанности:
https://spb.hh.ru/vacancy/36464

https://spb.hh.ru/vacancy/36494644
https://spb.hh.ru/vacancy/36488823
https://spb.hh.ru/vacancy/32881404
https://spb.hh.ru/vacancy/36486372
обязанности:
https://spb.hh.ru/vacancy/36226256
https://spb.hh.ru/vacancy/36070153
https://spb.hh.ru/vacancy/36151870
https://spb.hh.ru/vacancy/36488636
обязанности:
https://spb.hh.ru/vacancy/36470956
https://spb.hh.ru/vacancy/36489944
обязанности:
https://spb.hh.ru/vacancy/36477829
https://spb.hh.ru/vacancy/36070149
обязанности:
https://spb.hh.ru/vacancy/36440357
https://spb.hh.ru/vacancy/36282820
https://spb.hh.ru/vacancy/36489941
https://spb.hh.ru/vacancy/36479242
https://spb.hh.ru/vacancy/32881669
https://spb.hh.ru/vacancy/35658246
https://spb.hh.ru/vacancy/36477353
https://spb.hh.ru/vacancy/35695754
https://spb.hh.ru/vacancy/36031966
https://spb.hh.ru/vacancy/36485463
https://spb.hh.ru/vacancy/33692102
https://spb.hh.ru/vacancy/35928043
обязанности
https://spb.hh.ru/vacancy/35512378
https://spb.hh.ru/vacancy/36484951
https://spb.hh.ru/vacancy/

In [ ]:

dat.to_csv(r'Z:\Collecting_data_openedu\2.csv')

In [7]:
data = get_vacancies(1, 'https://spb.hh.ru/catalog/Administrativnyj-personal')
data.head()

NameError: name 'links_to_courses' is not defined

In [25]:
data  = get_vacancies(35, 'https://spb.hh.ru/catalog/Banki-Investicii-Lizing')
data.to_csv(r'Z:\Collecting_data_openedu\4.csv')

обязанности:
https://spb.hh.ru/vacancy/35830829
обязанности:
https://spb.hh.ru/vacancy/35674791
обязанности:
https://spb.hh.ru/vacancy/36270687
обязанности:
https://spb.hh.ru/vacancy/35830836
обязанности:
https://spb.hh.ru/vacancy/36365137
https://spb.hh.ru/vacancy/36266683
https://spb.hh.ru/vacancy/36501656
https://spb.hh.ru/vacancy/34786132
обязанности:
https://spb.hh.ru/vacancy/36129243
https://spb.hh.ru/vacancy/36348253
https://spb.hh.ru/vacancy/36155292
https://spb.hh.ru/vacancy/35648473
обязанности:
https://spb.hh.ru/vacancy/36527000
обязанности:
https://spb.hh.ru/vacancy/36315807
https://spb.hh.ru/vacancy/36271827
https://spb.hh.ru/vacancy/33369279
https://spb.hh.ru/vacancy/36302155
https://spb.hh.ru/vacancy/35444033
ваши обязанности
https://spb.hh.ru/vacancy/36415837
обязанности:
https://spb.hh.ru/vacancy/36109635
https://spb.hh.ru/vacancy/33182541
обязанности:
https://spb.hh.ru/vacancy/34477888
обязанности: 1 линия поддержки
https://spb.hh.ru/vacancy/36230699
обязанности:
http

https://spb.hh.ru/vacancy/36357337
https://spb.hh.ru/vacancy/36390100
обязанности:
https://spb.hh.ru/vacancy/36281999
https://spb.hh.ru/vacancy/36408988
обязанности:
https://spb.hh.ru/vacancy/36379983
https://spb.hh.ru/vacancy/36198062
обязанности:
https://spb.hh.ru/vacancy/36074849
https://spb.hh.ru/vacancy/36359708
обязанности:
https://spb.hh.ru/vacancy/36097399
обязанности
https://spb.hh.ru/vacancy/36406199
https://spb.hh.ru/vacancy/36162066
обязанности:
https://spb.hh.ru/vacancy/36060519
https://spb.hh.ru/vacancy/36113150
обязанности:
https://spb.hh.ru/vacancy/36236843
обязанности:
https://spb.hh.ru/vacancy/35649983
https://spb.hh.ru/vacancy/36437647
обязанности:
https://spb.hh.ru/vacancy/36384670
обязанности:
https://spb.hh.ru/vacancy/35650100
обязанности:
https://spb.hh.ru/vacancy/36448456
обязанности:
https://spb.hh.ru/vacancy/36200113
обязанности:
https://spb.hh.ru/vacancy/36117956
обязанности:
https://spb.hh.ru/vacancy/36316860
обязанности :
https://spb.hh.ru/vacancy/36219908


обязанности:
https://spb.hh.ru/vacancy/36519839
обязанности:
https://spb.hh.ru/vacancy/35627781
обязанности:
https://spb.hh.ru/vacancy/36244657
обязанности:
https://spb.hh.ru/vacancy/36100748
обязанности:
https://spb.hh.ru/vacancy/36307698
https://spb.hh.ru/vacancy/36337931
обязанности:
https://spb.hh.ru/vacancy/36332859
обязанности:
https://spb.hh.ru/vacancy/36156208
должностные обязанности:
https://spb.hh.ru/vacancy/36371705
обязанности:
https://spb.hh.ru/vacancy/35920733
https://spb.hh.ru/vacancy/36166183
обязанности:
https://spb.hh.ru/vacancy/36153189
обязанности:
https://spb.hh.ru/vacancy/36391554
обязанности: 
https://spb.hh.ru/vacancy/36466065
обязанности:
https://spb.hh.ru/vacancy/36281539
обязанности:
https://spb.hh.ru/vacancy/36466406
обязанности:
https://spb.hh.ru/vacancy/36110135
обязанности:
https://spb.hh.ru/vacancy/36500163
обязанности:
https://spb.hh.ru/vacancy/36244478
обязанности:
https://spb.hh.ru/vacancy/36123541
https://spb.hh.ru/vacancy/35724852
обязанности:
https

обязанности:
https://spb.hh.ru/vacancy/35722622
обязанности:
https://spb.hh.ru/vacancy/36290099
обязанности:
https://spb.hh.ru/vacancy/36145538
https://spb.hh.ru/vacancy/35832943
обязанности:
https://spb.hh.ru/vacancy/35962609
https://spb.hh.ru/vacancy/36293068
обязанности:
https://spb.hh.ru/vacancy/36423273
обязанности:
https://spb.hh.ru/vacancy/35463440
https://spb.hh.ru/vacancy/36367389
обязанности:
https://spb.hh.ru/vacancy/36405594
https://spb.hh.ru/vacancy/36519586
обязанности:
https://spb.hh.ru/vacancy/36325131
обязанности:
https://spb.hh.ru/vacancy/36367909
обязанности:
https://spb.hh.ru/vacancy/36409527
обязанности:
https://spb.hh.ru/vacancy/36363033
обязанности:
https://spb.hh.ru/vacancy/36414407
https://spb.hh.ru/vacancy/35844224
обязанности:
https://spb.hh.ru/vacancy/36065581
https://spb.hh.ru/vacancy/36369729
должностные обязанности:
https://spb.hh.ru/vacancy/36339534
обязанности:
https://spb.hh.ru/vacancy/36212696
обязанности:
https://spb.hh.ru/vacancy/36154023
обязанност

https://spb.hh.ru/vacancy/36518897
должностные обязанности:
https://spb.hh.ru/vacancy/35802922
обязанности:
https://spb.hh.ru/vacancy/35648066
https://spb.hh.ru/vacancy/36261947
обязанности:
https://spb.hh.ru/vacancy/36392690
https://spb.hh.ru/vacancy/36456978
https://spb.hh.ru/vacancy/35237272
https://spb.hh.ru/vacancy/36167102
обязанности:
https://spb.hh.ru/vacancy/36105228
обязанности:
https://spb.hh.ru/vacancy/36480555
обязанности:
https://spb.hh.ru/vacancy/35189331
https://spb.hh.ru/vacancy/35791568
обязанности:
https://spb.hh.ru/vacancy/36124119
обязанности:
https://spb.hh.ru/vacancy/36429404
обязанности:
https://spb.hh.ru/vacancy/35128522
https://spb.hh.ru/vacancy/36367493
https://spb.hh.ru/vacancy/32438629
обязанности:
https://spb.hh.ru/vacancy/35190064
https://spb.hh.ru/vacancy/36436985
обязанности:
https://spb.hh.ru/vacancy/36279181
обязанности:
https://spb.hh.ru/vacancy/36277296
https://spb.hh.ru/vacancy/36316215
https://spb.hh.ru/vacancy/36266021
обязанности:
https://spb.hh

обязанности:
https://spb.hh.ru/vacancy/35478024
обязанности:
https://spb.hh.ru/vacancy/36311306
обязанности:
https://spb.hh.ru/vacancy/36114861
обязанности:
https://spb.hh.ru/vacancy/35874450
обязанности:
https://spb.hh.ru/vacancy/35286517
https://spb.hh.ru/vacancy/36277651
https://spb.hh.ru/vacancy/35406009
обязанности
https://spb.hh.ru/vacancy/36154062
https://spb.hh.ru/vacancy/34890596
обязанности:
https://spb.hh.ru/vacancy/35409193
https://spb.hh.ru/vacancy/36121332
обязанности:
https://spb.hh.ru/vacancy/34340254
обязанности:
https://spb.hh.ru/vacancy/35928766
обязанности:
https://spb.hh.ru/vacancy/36230030
обязанности:
https://spb.hh.ru/vacancy/35410089
обязанности:
https://spb.hh.ru/vacancy/34887594
обязанности: 
https://spb.hh.ru/vacancy/35202072
обязанности:
https://spb.hh.ru/vacancy/36203729
обязанности:
https://spb.hh.ru/vacancy/36231851
https://spb.hh.ru/vacancy/35279166
https://spb.hh.ru/vacancy/36145130
обязанности:
https://spb.hh.ru/vacancy/35998745
https://spb.hh.ru/vaca

обязанности:
https://spb.hh.ru/vacancy/36017284
обязанности:
https://spb.hh.ru/vacancy/36305202
обязанности:
https://spb.hh.ru/vacancy/36231531
https://spb.hh.ru/vacancy/30001574
обязанности:
https://spb.hh.ru/vacancy/36204900
обязанности:
https://spb.hh.ru/vacancy/35213358
обязанности:
https://spb.hh.ru/vacancy/34210760
обязанности:
https://spb.hh.ru/vacancy/36238099
обязанности:
https://spb.hh.ru/vacancy/35676732
обязанности:
https://spb.hh.ru/vacancy/36176176
обязанности:
https://spb.hh.ru/vacancy/14526956
https://spb.hh.ru/vacancy/30963449
обязанности:
https://spb.hh.ru/vacancy/36180140
обязанности:
https://spb.hh.ru/vacancy/35793802
обязанности:
https://spb.hh.ru/vacancy/36243106
обязанности
https://spb.hh.ru/vacancy/35151614
https://spb.hh.ru/vacancy/36055620
обязанности:
https://spb.hh.ru/vacancy/36476382
обязанности:
https://spb.hh.ru/vacancy/36237357
обязанности:
https://spb.hh.ru/vacancy/36367434
https://spb.hh.ru/vacancy/36326020
обязанности:
https://spb.hh.ru/vacancy/356261

https://spb.hh.ru/vacancy/35985132
https://spb.hh.ru/vacancy/33436360
обязанности:
https://spb.hh.ru/vacancy/32499323
обязанности:
https://spb.hh.ru/vacancy/34093080
обязанности:
https://spb.hh.ru/vacancy/31697391
https://spb.hh.ru/vacancy/36233007
обязанности:
https://spb.hh.ru/vacancy/33105319
обязанности:
https://spb.hh.ru/vacancy/36386107
обязанности
https://spb.hh.ru/vacancy/35653639
обязанности:
https://spb.hh.ru/vacancy/35869714
обязанности:
https://spb.hh.ru/vacancy/36241081
обязанности:
https://spb.hh.ru/vacancy/36167290
https://spb.hh.ru/vacancy/31085790
обязанности:
https://spb.hh.ru/vacancy/35258931
обязанности:
https://spb.hh.ru/vacancy/35149673
https://spb.hh.ru/vacancy/35790011
обязанности: 
https://spb.hh.ru/vacancy/33712461
https://spb.hh.ru/vacancy/30309672
обязанности:
https://spb.hh.ru/vacancy/36312128
обязанности:
https://spb.hh.ru/vacancy/36373818
https://spb.hh.ru/vacancy/35444033
https://spb.hh.ru/vacancy/31062733
https://spb.hh.ru/vacancy/36358350
обязанности:


обязанности:
https://spb.hh.ru/vacancy/36389076
https://spb.hh.ru/vacancy/35019117
обязанности:
https://spb.hh.ru/vacancy/35621706
обязанности:
https://spb.hh.ru/vacancy/35823501
обязанности:
https://spb.hh.ru/vacancy/36003780
обязанности:
https://spb.hh.ru/vacancy/36058722
обязанности:
https://spb.hh.ru/vacancy/36059937
https://spb.hh.ru/vacancy/36082168
обязанности:
https://spb.hh.ru/vacancy/36114012
обязанности
https://spb.hh.ru/vacancy/36133237
обязанности:
https://spb.hh.ru/vacancy/36137130
https://spb.hh.ru/vacancy/36142848
обязанности:
https://spb.hh.ru/vacancy/36163608
обязанности:
https://spb.hh.ru/vacancy/36170181
https://spb.hh.ru/vacancy/36170259
обязанности:
https://spb.hh.ru/vacancy/36178897
обязанности:
https://spb.hh.ru/vacancy/36226087
обязанности:
https://spb.hh.ru/vacancy/36226530
обязанности:
https://spb.hh.ru/vacancy/36287565
обязанности:
https://spb.hh.ru/vacancy/36291797
обязанности:
https://spb.hh.ru/vacancy/36313408
должностные обязанности:
https://spb.hh.ru/va

In [26]:
data  = get_vacancies(40, 'https://spb.hh.ru/catalog/Buhgalterija-Upravlencheskij-uchet-Finansy-predprijatija')
data.to_csv(r'Z:\Collecting_data_openedu\5.csv')

обязанности:
https://spb.hh.ru/vacancy/36397226
обязанности:
https://spb.hh.ru/vacancy/36146777
обязанности:
https://spb.hh.ru/vacancy/35674791
https://spb.hh.ru/vacancy/36306403
https://spb.hh.ru/vacancy/36061580
https://spb.hh.ru/vacancy/36042232
обязанности:
https://spb.hh.ru/vacancy/36527000
обязанности:
https://spb.hh.ru/vacancy/35346900
https://spb.hh.ru/vacancy/36121862
https://spb.hh.ru/vacancy/36474454
https://spb.hh.ru/vacancy/36259725
https://spb.hh.ru/vacancy/36502153
https://spb.hh.ru/vacancy/35316953
https://spb.hh.ru/vacancy/33907008
обязанности:
https://spb.hh.ru/vacancy/35721079
https://spb.hh.ru/vacancy/36267910
https://spb.hh.ru/vacancy/35470510
https://spb.hh.ru/vacancy/36499428
обязанности:
https://spb.hh.ru/vacancy/36227962
https://spb.hh.ru/vacancy/36474097
обязанности:
https://spb.hh.ru/vacancy/36241145
обязанности:
https://spb.hh.ru/vacancy/36326166
https://spb.hh.ru/vacancy/36238166
обязанности:
https://spb.hh.ru/vacancy/36157991
https://spb.hh.ru/vacancy/3606

обязанности:
https://spb.hh.ru/vacancy/36314102
обязанности:
https://spb.hh.ru/vacancy/36443693
обязанности:
https://spb.hh.ru/vacancy/36465495
обязанности:
https://spb.hh.ru/vacancy/36444514
обязанности:
https://spb.hh.ru/vacancy/36093957
обязанности:
https://spb.hh.ru/vacancy/36431208
обязанности:
https://spb.hh.ru/vacancy/36465412
обязанности:
https://spb.hh.ru/vacancy/36429048
обязанности:
https://spb.hh.ru/vacancy/36408592
обязанности:
https://spb.hh.ru/vacancy/36142230
обязанности:
https://spb.hh.ru/vacancy/36405512
обязанности:
https://spb.hh.ru/vacancy/36191470
https://spb.hh.ru/vacancy/36451404
обязанности:
https://spb.hh.ru/vacancy/36406294
обязанности:
https://spb.hh.ru/vacancy/36202994
обязанности:
https://spb.hh.ru/vacancy/35876970
https://spb.hh.ru/vacancy/36295698
https://spb.hh.ru/vacancy/36405809
обязанности:
https://spb.hh.ru/vacancy/36446014
https://spb.hh.ru/vacancy/36313893
обязанности:
https://spb.hh.ru/vacancy/36447633
обязанности:
https://spb.hh.ru/vacancy/36447

обязанности:
https://spb.hh.ru/vacancy/35709799
обязанности:
https://spb.hh.ru/vacancy/36453692
обязанности:
https://spb.hh.ru/vacancy/36519476
https://spb.hh.ru/vacancy/36375014
обязанности:
https://spb.hh.ru/vacancy/36393921
обязанности:
https://spb.hh.ru/vacancy/36282842
https://spb.hh.ru/vacancy/36239301
обязанности:
https://spb.hh.ru/vacancy/36254835
обязанности:
https://spb.hh.ru/vacancy/36290847
обязанности:
https://spb.hh.ru/vacancy/36479038
обязанности:
https://spb.hh.ru/vacancy/36290676
обязанности:
https://spb.hh.ru/vacancy/36501746
https://spb.hh.ru/vacancy/35716689
обязанности:
https://spb.hh.ru/vacancy/36319882
обязанности:
https://spb.hh.ru/vacancy/36499337
обязанности:
https://spb.hh.ru/vacancy/36346934
обязанности:
https://spb.hh.ru/vacancy/36206287
обязанности:
https://spb.hh.ru/vacancy/36234707
https://spb.hh.ru/vacancy/36235019
https://spb.hh.ru/vacancy/36335172
ключевые обязанности: 
https://spb.hh.ru/vacancy/36403571
должностные обязанности:
https://spb.hh.ru/vaca

обязанности:
https://spb.hh.ru/vacancy/36355819
обязанности:
https://spb.hh.ru/vacancy/35155119
https://spb.hh.ru/vacancy/36345828
обязанности
https://spb.hh.ru/vacancy/35896861
https://spb.hh.ru/vacancy/36318349
обязанности:
https://spb.hh.ru/vacancy/35743360
обязанности:
https://spb.hh.ru/vacancy/36163664
в ваши обязанности будет входить:
https://spb.hh.ru/vacancy/36257700
https://spb.hh.ru/vacancy/36345448
обязанности:
https://spb.hh.ru/vacancy/35947481
https://spb.hh.ru/vacancy/36173857
обязанности:
https://spb.hh.ru/vacancy/36417924
обязанности:
https://spb.hh.ru/vacancy/36350248
обязанности:
https://spb.hh.ru/vacancy/34702189
обязанности:
https://spb.hh.ru/vacancy/35943688
обязанности:
https://spb.hh.ru/vacancy/36042477
обязанности:
https://spb.hh.ru/vacancy/36336888
обязанности:
https://spb.hh.ru/vacancy/36338203
https://spb.hh.ru/vacancy/35894355
обязанности:
https://spb.hh.ru/vacancy/36243009
обязанности:
https://spb.hh.ru/vacancy/36306988
https://spb.hh.ru/vacancy/36335172
об

обязанности:
https://spb.hh.ru/vacancy/36114182
обязанности:
https://spb.hh.ru/vacancy/35650511
обязанности:
https://spb.hh.ru/vacancy/36094650
https://spb.hh.ru/vacancy/36295299
https://spb.hh.ru/vacancy/36115335
обязанности:
https://spb.hh.ru/vacancy/36093957
обязанности:
https://spb.hh.ru/vacancy/36140438
обязанности:
https://spb.hh.ru/vacancy/36134425
обязанности: 
https://spb.hh.ru/vacancy/36233751
обязанности:
https://spb.hh.ru/vacancy/36139293
обязанности
https://spb.hh.ru/vacancy/35653639
https://spb.hh.ru/vacancy/36130568
обязанности:
https://spb.hh.ru/vacancy/35967023
обязанности:
https://spb.hh.ru/vacancy/34841610
обязанности:
https://spb.hh.ru/vacancy/36111914
обязанности:
https://spb.hh.ru/vacancy/34097537
обязанности:
https://spb.hh.ru/vacancy/36123968
обязанности:
https://spb.hh.ru/vacancy/36108308
обязанности:
https://spb.hh.ru/vacancy/36180790
https://spb.hh.ru/vacancy/36090209
обязанности:
https://spb.hh.ru/vacancy/36112280
обязанности:
https://spb.hh.ru/vacancy/36122

...

обязанности:
https://spb.hh.ru/vacancy/35447705
обязанности:
https://spb.hh.ru/vacancy/36329815
в ваши обязанности будет входить:
https://spb.hh.ru/vacancy/36159006
обязанности:
https://spb.hh.ru/vacancy/35305918
обязанности:
https://spb.hh.ru/vacancy/36153687
обязанности:
https://spb.hh.ru/vacancy/36404565
обязанности:
https://spb.hh.ru/vacancy/36351464
обязанности:
https://spb.hh.ru/vacancy/36276226
обязанности:
https://spb.hh.ru/vacancy/36459729
обязанности:
https://spb.hh.ru/vacancy/36461541
обязанности:
https://spb.hh.ru/vacancy/36169034
обязанности:
https://spb.hh.ru/vacancy/36196730
https://spb.hh.ru/vacancy/36167834
обязанности:
https://spb.hh.ru/vacancy/36349454
обязанности:
https://spb.hh.ru/vacancy/36415340
обязанности:
https://spb.hh.ru/vacancy/36242983
обязанности:
https://spb.hh.ru/vacancy/36333740
обязанности:
https://spb.hh.ru/vacancy/36452144
обязанности: 
https://spb.hh.ru/vacancy/35411544
обязанности:
https://spb.hh.ru/vacancy/36203832
обязанности:
https://spb.hh.ru/

https://spb.hh.ru/vacancy/36365013
обязанности:
https://spb.hh.ru/vacancy/34099857
обязанности:
https://spb.hh.ru/vacancy/35998745
обязанности:
https://spb.hh.ru/vacancy/35950098
обязанности:
https://spb.hh.ru/vacancy/35967719
обязанности:
https://spb.hh.ru/vacancy/36486599
должностные обязанности:
https://spb.hh.ru/vacancy/36027426
https://spb.hh.ru/vacancy/35937506
https://spb.hh.ru/vacancy/35816859
обязанности:
https://spb.hh.ru/vacancy/36098272
обязанности:
https://spb.hh.ru/vacancy/36240806
обязанности:
https://spb.hh.ru/vacancy/36211528
обязанности:
https://spb.hh.ru/vacancy/36325996
обязанности:
https://spb.hh.ru/vacancy/35107897
обязанности: 
https://spb.hh.ru/vacancy/35134600
обязанности:
https://spb.hh.ru/vacancy/35529138
обязанности
https://spb.hh.ru/vacancy/36403683
обязанности
https://spb.hh.ru/vacancy/35787888
обязанности:
https://spb.hh.ru/vacancy/36330740
обязанности:
https://spb.hh.ru/vacancy/36316923
https://spb.hh.ru/vacancy/36257480
обязанности:
https://spb.hh.ru/va

обязанности:
https://spb.hh.ru/vacancy/36392695
https://spb.hh.ru/vacancy/36296617
обязанности:
https://spb.hh.ru/vacancy/35146169
обязанности:
https://spb.hh.ru/vacancy/36367434
https://spb.hh.ru/vacancy/36145610
https://spb.hh.ru/vacancy/36326020
https://spb.hh.ru/vacancy/36112132
https://spb.hh.ru/vacancy/36444199
обязанности:
https://spb.hh.ru/vacancy/35849840
https://spb.hh.ru/vacancy/36296907
обязанности:
https://spb.hh.ru/vacancy/36169147
обязанности:
https://spb.hh.ru/vacancy/36240775
обязанности:
https://spb.hh.ru/vacancy/36334029
обязанности:
https://spb.hh.ru/vacancy/36431771
обязанности:
https://spb.hh.ru/vacancy/36211086
обязанности:
https://spb.hh.ru/vacancy/36229525
https://spb.hh.ru/vacancy/36172070
https://spb.hh.ru/vacancy/36512198
должностные обязанности:
https://spb.hh.ru/vacancy/36129142
обязанности:
https://spb.hh.ru/vacancy/36384603
обязанности:
https://spb.hh.ru/vacancy/36431598
обязанности:
https://spb.hh.ru/vacancy/36123104
обязанности:
https://spb.hh.ru/vacan

обязанности:
https://spb.hh.ru/vacancy/36461541
обязанности:
https://spb.hh.ru/vacancy/36169034
https://spb.hh.ru/vacancy/36167834
обязанности:
https://spb.hh.ru/vacancy/36452144
https://spb.hh.ru/vacancy/35730752
https://spb.hh.ru/vacancy/36214150
обязанности:
https://spb.hh.ru/vacancy/36355188
https://spb.hh.ru/vacancy/36380142
обязанности: 
https://spb.hh.ru/vacancy/36198873
https://spb.hh.ru/vacancy/33124893
обязанности:
https://spb.hh.ru/vacancy/36509674
https://spb.hh.ru/vacancy/36304090
https://spb.hh.ru/vacancy/36112260
обязанности:
https://spb.hh.ru/vacancy/36421858
обязанности:
https://spb.hh.ru/vacancy/35830855
https://spb.hh.ru/vacancy/33880394
https://spb.hh.ru/vacancy/36288117
обязанности:
https://spb.hh.ru/vacancy/36429810
профессиональные обязанности:
https://spb.hh.ru/vacancy/36209780
обязанности:
https://spb.hh.ru/vacancy/36045038
https://spb.hh.ru/vacancy/36227764
https://spb.hh.ru/vacancy/36166176
обязанности:
https://spb.hh.ru/vacancy/35673152
обязанности:
https://

должностные обязанности:
https://spb.hh.ru/vacancy/31669791
https://spb.hh.ru/vacancy/36054151
https://spb.hh.ru/vacancy/36064250
обязанности:
https://spb.hh.ru/vacancy/34282830
обязанности:
https://spb.hh.ru/vacancy/36272228
должностные обязанности:
https://spb.hh.ru/vacancy/35912493
https://spb.hh.ru/vacancy/36310824
обязанности:
https://spb.hh.ru/vacancy/36379114
https://spb.hh.ru/vacancy/36273745


In [27]:
data  = get_vacancies(16, 'https://spb.hh.ru/catalog/Vysshij-menedzhment')
data.to_csv(r'Z:\Collecting_data_openedu\6.csv')

обязанности:
https://spb.hh.ru/vacancy/36150609
https://spb.hh.ru/vacancy/35885602
обязанности
https://spb.hh.ru/vacancy/36494986
https://spb.hh.ru/vacancy/35656806
https://spb.hh.ru/vacancy/35715672
https://spb.hh.ru/vacancy/36378179
обязанности:
https://spb.hh.ru/vacancy/36321295
https://spb.hh.ru/vacancy/35854546
https://spb.hh.ru/vacancy/36317019
обязанности:
https://spb.hh.ru/vacancy/35558179
обязанности:
https://spb.hh.ru/vacancy/36311279
https://spb.hh.ru/vacancy/35871910
https://spb.hh.ru/vacancy/36244213
обязанности:
https://spb.hh.ru/vacancy/35557928
https://spb.hh.ru/vacancy/35977827
обязанности:
https://spb.hh.ru/vacancy/36449199
https://spb.hh.ru/vacancy/35769342
обязанности:
https://spb.hh.ru/vacancy/36372567
обязанности:
https://spb.hh.ru/vacancy/35757167
обязанности:
https://spb.hh.ru/vacancy/36527552
https://spb.hh.ru/vacancy/36337477
обязанности:
https://spb.hh.ru/vacancy/36139925
обязанности:
https://spb.hh.ru/vacancy/36271299
https://spb.hh.ru/vacancy/36199741
обяза

обязанности:
https://spb.hh.ru/vacancy/36297675
https://spb.hh.ru/vacancy/35943626
обязанности:
https://spb.hh.ru/vacancy/36124029
обязанности:
https://spb.hh.ru/vacancy/36124229
обязанности:
https://spb.hh.ru/vacancy/36124119
https://spb.hh.ru/vacancy/35655305
обязанности:
https://spb.hh.ru/vacancy/36123883
обязанности:
https://spb.hh.ru/vacancy/35965356
https://spb.hh.ru/vacancy/35549151
https://spb.hh.ru/vacancy/36181946
обязанности:
https://spb.hh.ru/vacancy/36163359
обязанности:
https://spb.hh.ru/vacancy/36118562
обязанности:
https://spb.hh.ru/vacancy/36176166
https://spb.hh.ru/vacancy/35833293
обязанности:
https://spb.hh.ru/vacancy/36089532
в ваши обязанности будет входить:
https://spb.hh.ru/vacancy/36136060
https://spb.hh.ru/vacancy/36124290
обязанности:
https://spb.hh.ru/vacancy/36163012
https://spb.hh.ru/vacancy/36100200
обязанности на данной должности:
https://spb.hh.ru/vacancy/36025076
https://spb.hh.ru/vacancy/36117225
https://spb.hh.ru/vacancy/36084203
обязанности:
https:/

обязанности:
https://spb.hh.ru/vacancy/36327437
https://spb.hh.ru/vacancy/35966249
https://spb.hh.ru/vacancy/36233542
обязанности:
https://spb.hh.ru/vacancy/36319621
обязанности
https://spb.hh.ru/vacancy/35351432
https://spb.hh.ru/vacancy/36243740
обязанности:
https://spb.hh.ru/vacancy/36300008
обязанности:
https://spb.hh.ru/vacancy/36414277
https://spb.hh.ru/vacancy/36285611
обязанности:
https://spb.hh.ru/vacancy/36168379
обязанности:
https://spb.hh.ru/vacancy/35939095
https://spb.hh.ru/vacancy/36306276
обязанности:
https://spb.hh.ru/vacancy/35999282
обязанности:
https://spb.hh.ru/vacancy/36319861
обязанности:
https://spb.hh.ru/vacancy/35846129
обязанности:
https://spb.hh.ru/vacancy/36281507
обязанности:
https://spb.hh.ru/vacancy/36258579
обязанности:
https://spb.hh.ru/vacancy/36179842
обязанности:
https://spb.hh.ru/vacancy/36066063
обязанности:
https://spb.hh.ru/vacancy/36042414
в ваши обязанности будет входить:
https://spb.hh.ru/vacancy/36437070
обязанности:
https://spb.hh.ru/vacanc

https://spb.hh.ru/vacancy/35333829
обязанности:
https://spb.hh.ru/vacancy/36056494
https://spb.hh.ru/vacancy/36152225
обязанности:
https://spb.hh.ru/vacancy/35257998
обязанности:
https://spb.hh.ru/vacancy/35793640
https://spb.hh.ru/vacancy/36185772
https://spb.hh.ru/vacancy/35938927
обязанности:
https://spb.hh.ru/vacancy/36086295
https://spb.hh.ru/vacancy/36101528
обязанности:
https://spb.hh.ru/vacancy/36376270
обязанности:
https://spb.hh.ru/vacancy/36357234
https://spb.hh.ru/vacancy/36075020
обязанности:
https://spb.hh.ru/vacancy/34834039
профессиональные обязанности:
https://spb.hh.ru/vacancy/35772920
https://spb.hh.ru/vacancy/36422786
обязанности:
https://spb.hh.ru/vacancy/35924391
https://spb.hh.ru/vacancy/36270911
обязанности:​​​​​​
https://spb.hh.ru/vacancy/36270943
обязанности:
https://spb.hh.ru/vacancy/35712034
https://spb.hh.ru/vacancy/36085385
обязанности:
https://spb.hh.ru/vacancy/36088057
должностные обязанности:
https://spb.hh.ru/vacancy/35482465
обязанности:
https://spb.h

In [28]:
data  = get_vacancies(4, 'https://spb.hh.ru/catalog/Gosudarstvennaja-sluzhba-nekommercheskie-organizacii')
data.to_csv(r'Z:\Collecting_data_openedu\7.csv')

обязанности:
https://spb.hh.ru/vacancy/26083602
https://spb.hh.ru/vacancy/36283782
https://spb.hh.ru/vacancy/36371204
обязанности:
https://spb.hh.ru/vacancy/36217424
обязанности:
https://spb.hh.ru/vacancy/36276518
обязанности:
https://spb.hh.ru/vacancy/36092131
обязанности:
https://spb.hh.ru/vacancy/36364400
должностные обязанности:
https://spb.hh.ru/vacancy/35145980
https://spb.hh.ru/vacancy/35133442
https://spb.hh.ru/vacancy/36390665
обязанности:
https://spb.hh.ru/vacancy/36268198
обязанности:
https://spb.hh.ru/vacancy/36516193
обязанности:
https://spb.hh.ru/vacancy/36462203
обязанности:
https://spb.hh.ru/vacancy/36168614
обязанности:
https://spb.hh.ru/vacancy/36061409
обязанности:
https://spb.hh.ru/vacancy/36061471
обязанности:
https://spb.hh.ru/vacancy/36049804
обязанности:
https://spb.hh.ru/vacancy/36279955
обязанности:
https://spb.hh.ru/vacancy/36478165
обязанности:
https://spb.hh.ru/vacancy/34273227
https://spb.hh.ru/vacancy/36133846
обязанности:
https://spb.hh.ru/vacancy/331558

In [29]:
data  = get_vacancies(9, 'https://spb.hh.ru/catalog/Installjacija-i-servis')
data.to_csv(r'Z:\Collecting_data_openedu\8.csv')

https://spb.hh.ru/vacancy/36300299
обязанности: 
https://spb.hh.ru/vacancy/36308877
https://spb.hh.ru/vacancy/36194391
обязанности:
https://spb.hh.ru/vacancy/36319419
обязанности:
https://spb.hh.ru/vacancy/36430642
обязанности:
https://spb.hh.ru/vacancy/36089786
обязанности:
https://spb.hh.ru/vacancy/36013026
обязанности:
https://spb.hh.ru/vacancy/36111700
обязанности:
https://spb.hh.ru/vacancy/36313101
https://spb.hh.ru/vacancy/34685428
обязанности:
https://spb.hh.ru/vacancy/36440298
https://spb.hh.ru/vacancy/35631843
обязанности:
https://spb.hh.ru/vacancy/36371177
обязанности:
https://spb.hh.ru/vacancy/36216535
обязанности:
https://spb.hh.ru/vacancy/36508753
обязанности:
https://spb.hh.ru/vacancy/36354204
https://spb.hh.ru/vacancy/34969175
обязанности:
https://spb.hh.ru/vacancy/36283356
https://spb.hh.ru/vacancy/33369279
обязанности инженера:
https://spb.hh.ru/vacancy/36180721
https://spb.hh.ru/vacancy/36266314
обязанности:
https://spb.hh.ru/vacancy/36000642
https://spb.hh.ru/vacancy

обязанности:
https://spb.hh.ru/vacancy/36231255
обязанности:
https://spb.hh.ru/vacancy/36313101
обязанности:
https://spb.hh.ru/vacancy/36286630
https://spb.hh.ru/vacancy/35918551
обязанности:
https://spb.hh.ru/vacancy/36232565
https://spb.hh.ru/vacancy/36217225
обязанности:
https://spb.hh.ru/vacancy/35242158
обязанности: 
https://spb.hh.ru/vacancy/33901213
https://spb.hh.ru/vacancy/36269933
обязанности:
https://spb.hh.ru/vacancy/36472931
обязанности:
https://spb.hh.ru/vacancy/36394926
обязанности:
https://spb.hh.ru/vacancy/36313872
обязанности:
https://spb.hh.ru/vacancy/36065613
обязанности:
https://spb.hh.ru/vacancy/36291761
https://spb.hh.ru/vacancy/35967417
https://spb.hh.ru/vacancy/34840503
обязанности:
https://spb.hh.ru/vacancy/35694178
обязанности:
https://spb.hh.ru/vacancy/36355015
https://spb.hh.ru/vacancy/35908868
https://spb.hh.ru/vacancy/36100149
обязанности:
https://spb.hh.ru/vacancy/36392088
должностные обязанности:
https://spb.hh.ru/vacancy/36369786
обязанности:
https://s

обязанности:
https://spb.hh.ru/vacancy/36163843
обязанности: 
https://spb.hh.ru/vacancy/36150829
обязанности:
https://spb.hh.ru/vacancy/36121220
https://spb.hh.ru/vacancy/34304043
основные обязанности:
https://spb.hh.ru/vacancy/36058666
обязанности:
https://spb.hh.ru/vacancy/36237612
обязанности:
https://spb.hh.ru/vacancy/36083529
https://spb.hh.ru/vacancy/36091931
https://spb.hh.ru/vacancy/36131419
обязанности:
https://spb.hh.ru/vacancy/35749216


In [30]:
data  = get_vacancies(17, 'https://spb.hh.ru/catalog/Iskusstvo-Razvlechenija-Mass-media')
data.to_csv(r'Z:\Collecting_data_openedu\9.csv')

обязанности:
https://spb.hh.ru/vacancy/36271867
https://spb.hh.ru/vacancy/31672284
https://spb.hh.ru/vacancy/36296930
мы — команда издательского контента проекта world of warships. мы думаем, что издательский контент — это проекты для диалога с игроками. мы не пишем тексты ради текстов и активно участвуем в разработке игры и маркетинговых активностях. и сейчас мы в поиске копирайтера, в обязанности которого будет входить:
https://spb.hh.ru/vacancy/36200565
https://spb.hh.ru/vacancy/31333042
https://spb.hh.ru/vacancy/36376437
https://spb.hh.ru/vacancy/32346259
https://spb.hh.ru/vacancy/36123061
в ваши обязанности будет входить выполнение следующих задач:
https://spb.hh.ru/vacancy/36196694
https://spb.hh.ru/vacancy/36162431
https://spb.hh.ru/vacancy/36213473
https://spb.hh.ru/vacancy/36443652
обязанности:
https://spb.hh.ru/vacancy/36049564
https://spb.hh.ru/vacancy/36478423
https://spb.hh.ru/vacancy/36359256
обязанности:
https://spb.hh.ru/vacancy/36528573
https://spb.hh.ru/vacancy/364489

https://spb.hh.ru/vacancy/36211834
https://spb.hh.ru/vacancy/35673561
https://spb.hh.ru/vacancy/36191419
https://spb.hh.ru/vacancy/36187512
https://spb.hh.ru/vacancy/36168979
https://spb.hh.ru/vacancy/35772840
обязанности:
https://spb.hh.ru/vacancy/36287544
https://spb.hh.ru/vacancy/36263104
обязанности:
https://spb.hh.ru/vacancy/36163891
https://spb.hh.ru/vacancy/36150911
https://spb.hh.ru/vacancy/36270765
https://spb.hh.ru/vacancy/36081637
обязанности:
https://spb.hh.ru/vacancy/35803368
обязанности:
https://spb.hh.ru/vacancy/30911344
обязанности:
https://spb.hh.ru/vacancy/36216174
https://spb.hh.ru/vacancy/33889994
https://spb.hh.ru/vacancy/36110850
https://spb.hh.ru/vacancy/36292746
обязанности:
https://spb.hh.ru/vacancy/36241734
обязанности:
https://spb.hh.ru/vacancy/36101987
https://spb.hh.ru/vacancy/36133941
обязанности:
https://spb.hh.ru/vacancy/36198980
https://spb.hh.ru/vacancy/36112707
https://spb.hh.ru/vacancy/36204833
https://spb.hh.ru/vacancy/36114821
обязанности:
https://

https://spb.hh.ru/vacancy/36341495
https://spb.hh.ru/vacancy/36026441
должностные обязанности:
https://spb.hh.ru/vacancy/36503945
обязанности:
https://spb.hh.ru/vacancy/36230744
https://spb.hh.ru/vacancy/36147058
https://spb.hh.ru/vacancy/35865991
https://spb.hh.ru/vacancy/36308019
обязанности:
https://spb.hh.ru/vacancy/36062944
обязанности:
https://spb.hh.ru/vacancy/36165375
обязанности:
https://spb.hh.ru/vacancy/36295273
обязанности:
https://spb.hh.ru/vacancy/36255975
https://spb.hh.ru/vacancy/35957265
https://spb.hh.ru/vacancy/36240046
https://spb.hh.ru/vacancy/34729345
обязанности:
https://spb.hh.ru/vacancy/35134048
обязанности:
https://spb.hh.ru/vacancy/36180685
https://spb.hh.ru/vacancy/36324431
обязанности:
https://spb.hh.ru/vacancy/36146582
https://spb.hh.ru/vacancy/36303520
обязанности:
https://spb.hh.ru/vacancy/35360291
https://spb.hh.ru/vacancy/36358573
https://spb.hh.ru/vacancy/35037312
обязанности:
https://spb.hh.ru/vacancy/34183967
https://spb.hh.ru/vacancy/36389802
https

https://spb.hh.ru/vacancy/34043779
обязанности:
https://spb.hh.ru/vacancy/35323918
https://spb.hh.ru/vacancy/32061697
https://spb.hh.ru/vacancy/36283171
https://spb.hh.ru/vacancy/36422786
обязанности:
https://spb.hh.ru/vacancy/34839436
https://spb.hh.ru/vacancy/33938642
https://spb.hh.ru/vacancy/36168808
https://spb.hh.ru/vacancy/36371355
https://spb.hh.ru/vacancy/34470687
https://spb.hh.ru/vacancy/36334564
обязанности:
https://spb.hh.ru/vacancy/36181655
обязанности:
https://spb.hh.ru/vacancy/36021103
профессиональные обязанности:
https://spb.hh.ru/vacancy/35846612
https://spb.hh.ru/vacancy/35856927
обязанности:
https://spb.hh.ru/vacancy/36121896
https://spb.hh.ru/vacancy/36217846
https://spb.hh.ru/vacancy/35638229
обязанности:
https://spb.hh.ru/vacancy/36021148
https://spb.hh.ru/vacancy/36356890
https://spb.hh.ru/vacancy/36373416
обязанности:
https://spb.hh.ru/vacancy/36417144
https://spb.hh.ru/vacancy/34627820
обязанности:
https://spb.hh.ru/vacancy/35589449
обязанности:
https://spb.h

обязанности:
https://spb.hh.ru/vacancy/36133393
https://spb.hh.ru/vacancy/36158875
обязанности:
https://spb.hh.ru/vacancy/36142344
обязанности
https://spb.hh.ru/vacancy/36083283
обязанности:
https://spb.hh.ru/vacancy/36051478
https://spb.hh.ru/vacancy/35713441


In [31]:
data  = get_vacancies(20, 'https://spb.hh.ru/catalog/Konsultirovanie')
data.to_csv(r'Z:\Collecting_data_openedu\10.csv')

https://spb.hh.ru/vacancy/35262030
обязанности:
https://spb.hh.ru/vacancy/36217443
https://spb.hh.ru/vacancy/36191145
https://spb.hh.ru/vacancy/36122483
https://spb.hh.ru/vacancy/31383178
https://spb.hh.ru/vacancy/34450089
https://spb.hh.ru/vacancy/36143050
https://spb.hh.ru/vacancy/36285579
https://spb.hh.ru/vacancy/36315664
https://spb.hh.ru/vacancy/31069771
https://spb.hh.ru/vacancy/36201686
обязанности:
https://spb.hh.ru/vacancy/36293270
обязанности:
https://spb.hh.ru/vacancy/35850748
https://spb.hh.ru/vacancy/36315197
https://spb.hh.ru/vacancy/36228726
обязанности:
https://spb.hh.ru/vacancy/36431727
https://spb.hh.ru/vacancy/36370795
https://spb.hh.ru/vacancy/36313557
https://spb.hh.ru/vacancy/36146259
обязанности:
https://spb.hh.ru/vacancy/36146957
https://spb.hh.ru/vacancy/36215410
https://spb.hh.ru/vacancy/36133261
https://spb.hh.ru/vacancy/36155292
обязанности:
https://spb.hh.ru/vacancy/34477888
https://spb.hh.ru/vacancy/35248455
https://spb.hh.ru/vacancy/36526209
https://spb.

обязанности:
https://spb.hh.ru/vacancy/35227174
обязанности :
https://spb.hh.ru/vacancy/36337935
https://spb.hh.ru/vacancy/35545907
обязанности:
https://spb.hh.ru/vacancy/36078257
обязанности:
https://spb.hh.ru/vacancy/36452673
https://spb.hh.ru/vacancy/36413592
обязанности:
https://spb.hh.ru/vacancy/35067303
обязанности:
https://spb.hh.ru/vacancy/36131182
обязанности:
https://spb.hh.ru/vacancy/36448460
https://spb.hh.ru/vacancy/36336196
https://spb.hh.ru/vacancy/36111133
https://spb.hh.ru/vacancy/27194521
https://spb.hh.ru/vacancy/34956739
https://spb.hh.ru/vacancy/34497951
https://spb.hh.ru/vacancy/35692211
обязанности:
https://spb.hh.ru/vacancy/36057346
https://spb.hh.ru/vacancy/34985632
обязанности:
https://spb.hh.ru/vacancy/36455322
обязанности:
https://spb.hh.ru/vacancy/36437871
обязанности:
https://spb.hh.ru/vacancy/35737611
обязанности:
https://spb.hh.ru/vacancy/36439833
обязанности :
https://spb.hh.ru/vacancy/36174996
обязанности:
https://spb.hh.ru/vacancy/35849951
обязанности

обязанности:
https://spb.hh.ru/vacancy/36320059
https://spb.hh.ru/vacancy/36356839
https://spb.hh.ru/vacancy/35528852
обязанности:
https://spb.hh.ru/vacancy/35470633
https://spb.hh.ru/vacancy/36327109
обязанности:
https://spb.hh.ru/vacancy/35721664
обязанности:
https://spb.hh.ru/vacancy/36365300
https://spb.hh.ru/vacancy/36356229
обязанности:
https://spb.hh.ru/vacancy/35920715
обязанности:
https://spb.hh.ru/vacancy/36326259
обязанности:
https://spb.hh.ru/vacancy/36315349
обязанности:
https://spb.hh.ru/vacancy/35293245
обязанности:
https://spb.hh.ru/vacancy/35013736
https://spb.hh.ru/vacancy/35988721
обязанности:
https://spb.hh.ru/vacancy/35953976
обязанности:
https://spb.hh.ru/vacancy/35850766
обязанности:
https://spb.hh.ru/vacancy/36359701
https://spb.hh.ru/vacancy/35653450
обязанности:
https://spb.hh.ru/vacancy/36351551
https://spb.hh.ru/vacancy/35936742
обязанности:
https://spb.hh.ru/vacancy/32312239
обязанности:
https://spb.hh.ru/vacancy/36333759
должностные обязанности:
https://sp

https://spb.hh.ru/vacancy/33653970
https://spb.hh.ru/vacancy/36052769
https://spb.hh.ru/vacancy/36071709
https://spb.hh.ru/vacancy/36060988
https://spb.hh.ru/vacancy/36072353
https://spb.hh.ru/vacancy/36054737
обязанности:
https://spb.hh.ru/vacancy/36121271
обязанности:
https://spb.hh.ru/vacancy/36169391
обязанности:
https://spb.hh.ru/vacancy/36024818
обязанности:
https://spb.hh.ru/vacancy/35121168
https://spb.hh.ru/vacancy/36063993
обязанности:
https://spb.hh.ru/vacancy/36062283
обязанности:
https://spb.hh.ru/vacancy/34872082
https://spb.hh.ru/vacancy/36061306
обязанности:
https://spb.hh.ru/vacancy/35540478
обязанности:
https://spb.hh.ru/vacancy/36160617
https://spb.hh.ru/vacancy/36147695
https://spb.hh.ru/vacancy/36143069
https://spb.hh.ru/vacancy/31383178
обязанности:
https://spb.hh.ru/vacancy/36253328
https://spb.hh.ru/vacancy/36141344
обязанности :
https://spb.hh.ru/vacancy/36143700
обязанности:
https://spb.hh.ru/vacancy/36236852
https://spb.hh.ru/vacancy/36263688
обязанности:
htt

https://spb.hh.ru/vacancy/33188197
https://spb.hh.ru/vacancy/34789410
https://spb.hh.ru/vacancy/36281965
https://spb.hh.ru/vacancy/36176740
обязанности:
https://spb.hh.ru/vacancy/36463485
https://spb.hh.ru/vacancy/33207739
обязанности:
https://spb.hh.ru/vacancy/36176868
основные обязанности :
https://spb.hh.ru/vacancy/36228197
https://spb.hh.ru/vacancy/33436360
обязанности:
https://spb.hh.ru/vacancy/36351570
обязанности:
https://spb.hh.ru/vacancy/36517925
https://spb.hh.ru/vacancy/35614309
https://spb.hh.ru/vacancy/36358193
https://spb.hh.ru/vacancy/27194521
обязанности:
https://spb.hh.ru/vacancy/36295165
https://spb.hh.ru/vacancy/35295370
https://spb.hh.ru/vacancy/35835695
https://spb.hh.ru/vacancy/36462998
https://spb.hh.ru/vacancy/36024649
https://spb.hh.ru/vacancy/36423514
https://spb.hh.ru/vacancy/33336412
https://spb.hh.ru/vacancy/35816301
https://spb.hh.ru/vacancy/35884636
https://spb.hh.ru/vacancy/34385894
https://spb.hh.ru/vacancy/35826779
обязанности:
https://spb.hh.ru/vacanc

In [32]:
data  = get_vacancies(40, 'https://spb.hh.ru/catalog/Marketing-Reklama-PR')
data.to_csv(r'Z:\Collecting_data_openedu\11.csv')

https://spb.hh.ru/vacancy/36523718
https://spb.hh.ru/vacancy/33269131
обязанности:
https://spb.hh.ru/vacancy/36509403
https://spb.hh.ru/vacancy/34577987
https://spb.hh.ru/vacancy/36466283
https://spb.hh.ru/vacancy/36449841
https://spb.hh.ru/vacancy/31672284
https://spb.hh.ru/vacancy/36449798
https://spb.hh.ru/vacancy/36143050
https://spb.hh.ru/vacancy/34688986
обязанности:
https://spb.hh.ru/vacancy/35277587
обязанности:
https://spb.hh.ru/vacancy/35998522
обязанности:
https://spb.hh.ru/vacancy/35660447
https://spb.hh.ru/vacancy/36042232
https://spb.hh.ru/vacancy/35648473
https://spb.hh.ru/vacancy/36313383
обязанности:
https://spb.hh.ru/vacancy/36412673
обязанности:
https://spb.hh.ru/vacancy/36288259
обязанности:
https://spb.hh.ru/vacancy/36311279
обязанности:
https://spb.hh.ru/vacancy/36217465
https://spb.hh.ru/vacancy/36183111
обязанности:
https://spb.hh.ru/vacancy/36147904
обязанности (в первое время обучим всему, далее будет специализация):
https://spb.hh.ru/vacancy/36204288
обязанно

https://spb.hh.ru/vacancy/36205284
обязанности:
https://spb.hh.ru/vacancy/36264770
https://spb.hh.ru/vacancy/34478285
обязанности:
https://spb.hh.ru/vacancy/36189299
https://spb.hh.ru/vacancy/35604873
https://spb.hh.ru/vacancy/36522698
https://spb.hh.ru/vacancy/35916088
https://spb.hh.ru/vacancy/34538572
обязанности:
https://spb.hh.ru/vacancy/36174471
обязанности:
https://spb.hh.ru/vacancy/36121524
https://spb.hh.ru/vacancy/36264255
https://spb.hh.ru/vacancy/34079628
https://spb.hh.ru/vacancy/36486556
обязанности:
https://spb.hh.ru/vacancy/36517332
https://spb.hh.ru/vacancy/36519845
обязанности:
https://spb.hh.ru/vacancy/35602912
обязанности:
https://spb.hh.ru/vacancy/36272343
https://spb.hh.ru/vacancy/36303635
обязанности:
https://spb.hh.ru/vacancy/36476900
https://spb.hh.ru/vacancy/36355579
обязанности:
https://spb.hh.ru/vacancy/36428762
https://spb.hh.ru/vacancy/36058204
https://spb.hh.ru/vacancy/36175943
https://spb.hh.ru/vacancy/36040994
обязанности:
https://spb.hh.ru/vacancy/3649

https://spb.hh.ru/vacancy/36092785
обязанности:
https://spb.hh.ru/vacancy/36495095
что будет входить в ваши обязанности?
https://spb.hh.ru/vacancy/36336956
https://spb.hh.ru/vacancy/36497228
https://spb.hh.ru/vacancy/36057405
https://spb.hh.ru/vacancy/35699401
обязанности:
https://spb.hh.ru/vacancy/36064259
https://spb.hh.ru/vacancy/36408877
обязанности:
https://spb.hh.ru/vacancy/36086183
обязанности:
https://spb.hh.ru/vacancy/36497362
https://spb.hh.ru/vacancy/36403419
https://spb.hh.ru/vacancy/36080090
https://spb.hh.ru/vacancy/35956709
https://spb.hh.ru/vacancy/36339256
обязанности:
https://spb.hh.ru/vacancy/36454304
https://spb.hh.ru/vacancy/36496595
https://spb.hh.ru/vacancy/36099878
должностные обязанности:
https://spb.hh.ru/vacancy/33594903
https://spb.hh.ru/vacancy/36187363
обязанности
https://spb.hh.ru/vacancy/36232486
обязанности:
https://spb.hh.ru/vacancy/35746081
https://spb.hh.ru/vacancy/36057159
обязанности (команды продвижения):
https://spb.hh.ru/vacancy/36391633
обязанн

обязанности:
https://spb.hh.ru/vacancy/36382721
https://spb.hh.ru/vacancy/36017278
обязанности:
https://spb.hh.ru/vacancy/36430127
обязанности:
https://spb.hh.ru/vacancy/36449637
обязанности:
https://spb.hh.ru/vacancy/36258851
https://spb.hh.ru/vacancy/34424977
https://spb.hh.ru/vacancy/35874038
https://spb.hh.ru/vacancy/36449476
обязанности:
https://spb.hh.ru/vacancy/36426374
https://spb.hh.ru/vacancy/36450745
https://spb.hh.ru/vacancy/36443859
обязанности:
https://spb.hh.ru/vacancy/36192041
https://spb.hh.ru/vacancy/36264303
обязанности:
https://spb.hh.ru/vacancy/36433092
должностные обязанности:
https://spb.hh.ru/vacancy/36426434
обязанности:
https://spb.hh.ru/vacancy/36426915
обязанности:
https://spb.hh.ru/vacancy/35201547
обязанности:
https://spb.hh.ru/vacancy/36438891
обязанности:
https://spb.hh.ru/vacancy/36270722
https://spb.hh.ru/vacancy/21963813
обязанности:
https://spb.hh.ru/vacancy/36293218
обязанности:
https://spb.hh.ru/vacancy/35838338
обязанности:
https://spb.hh.ru/vacan

https://spb.hh.ru/vacancy/36090252
https://spb.hh.ru/vacancy/36432690
обязанности:
https://spb.hh.ru/vacancy/36429838
https://spb.hh.ru/vacancy/36414968
https://spb.hh.ru/vacancy/36406152
https://spb.hh.ru/vacancy/35753270
обязанности:
https://spb.hh.ru/vacancy/36438891
обязанности и функции:
https://spb.hh.ru/vacancy/36075048
https://spb.hh.ru/vacancy/36415640
обязанности:
https://spb.hh.ru/vacancy/30298483
https://spb.hh.ru/vacancy/36421134
обязанности:
https://spb.hh.ru/vacancy/36196852
https://spb.hh.ru/vacancy/36415772
https://spb.hh.ru/vacancy/36429131
обязанности:
https://spb.hh.ru/vacancy/36411591
https://spb.hh.ru/vacancy/35549218
обязанности:
https://spb.hh.ru/vacancy/35880660
https://spb.hh.ru/vacancy/35347931
обязанности:
https://spb.hh.ru/vacancy/36419550
обязанности:
https://spb.hh.ru/vacancy/36388324
основные обязанности:
https://spb.hh.ru/vacancy/36410416
https://spb.hh.ru/vacancy/36292259
обязанности:
https://spb.hh.ru/vacancy/34210172
обязанности:
https://spb.hh.ru/va

...

https://spb.hh.ru/vacancy/36330840
https://spb.hh.ru/vacancy/36306221
обязанности:
https://spb.hh.ru/vacancy/36315583
обязанности:
https://spb.hh.ru/vacancy/33359221
обязанности:
https://spb.hh.ru/vacancy/36186288
обязанности:
https://spb.hh.ru/vacancy/36318550
https://spb.hh.ru/vacancy/36046551
https://spb.hh.ru/vacancy/36046567
https://spb.hh.ru/vacancy/36318179
обязанности:
https://spb.hh.ru/vacancy/35566493
https://spb.hh.ru/vacancy/36224590
https://spb.hh.ru/vacancy/36339813
https://spb.hh.ru/vacancy/36305894
https://spb.hh.ru/vacancy/36313445
https://spb.hh.ru/vacancy/36306948
обязанности:
https://spb.hh.ru/vacancy/36267684
обязанности:
https://spb.hh.ru/vacancy/36326485
что будет входить в обязанности:
https://spb.hh.ru/vacancy/36407281
https://spb.hh.ru/vacancy/35929079
https://spb.hh.ru/vacancy/35835015
https://spb.hh.ru/vacancy/35994012
https://spb.hh.ru/vacancy/36201892
https://spb.hh.ru/vacancy/35579452
https://spb.hh.ru/vacancy/36316703
обязанности:
https://spb.hh.ru/vacan

обязанности:
https://spb.hh.ru/vacancy/36257517
обязанности:
https://spb.hh.ru/vacancy/36236486
https://spb.hh.ru/vacancy/36192414
обязанности:
https://spb.hh.ru/vacancy/36320136
обязанности:
https://spb.hh.ru/vacancy/36066753
обязанности:
https://spb.hh.ru/vacancy/36143632
обязанности:
https://spb.hh.ru/vacancy/36244714
обязанности:
https://spb.hh.ru/vacancy/36207458
обязанности:
https://spb.hh.ru/vacancy/36319538
https://spb.hh.ru/vacancy/36231832
обязанности:
https://spb.hh.ru/vacancy/36204259
https://spb.hh.ru/vacancy/36237878
обязанности:
https://spb.hh.ru/vacancy/36068960
обязанности:
https://spb.hh.ru/vacancy/36196187
обязанности:
https://spb.hh.ru/vacancy/36205438
обязанности:
https://spb.hh.ru/vacancy/36212206
обязанности:
https://spb.hh.ru/vacancy/36211443
https://spb.hh.ru/vacancy/35452063
обязанности:
https://spb.hh.ru/vacancy/36224764
https://spb.hh.ru/vacancy/36227220
обязанности:
https://spb.hh.ru/vacancy/34641644
обязанности:
https://spb.hh.ru/vacancy/36172875
обязаннос

https://spb.hh.ru/vacancy/36123359
https://spb.hh.ru/vacancy/35721332
https://spb.hh.ru/vacancy/36112707
https://spb.hh.ru/vacancy/33465758
обязанности:
https://spb.hh.ru/vacancy/36057783
обязанности:
https://spb.hh.ru/vacancy/35794521
обязанности:от кандидата требуется пробный пресс-релиз
https://spb.hh.ru/vacancy/36086032
обязанности соискателя:
https://spb.hh.ru/vacancy/36142229
https://spb.hh.ru/vacancy/35141681
https://spb.hh.ru/vacancy/36261130
https://spb.hh.ru/vacancy/36120219
обязанности на данной должности:
https://spb.hh.ru/vacancy/36025076
https://spb.hh.ru/vacancy/35743945
https://spb.hh.ru/vacancy/35162158
обязанности:
https://spb.hh.ru/vacancy/36122548
обязанности:
https://spb.hh.ru/vacancy/36099483
https://spb.hh.ru/vacancy/35964378
обязанности:
https://spb.hh.ru/vacancy/36124829
обязанности:
https://spb.hh.ru/vacancy/35133741
https://spb.hh.ru/vacancy/34996627
https://spb.hh.ru/vacancy/36144788
https://spb.hh.ru/vacancy/36145310
обязанности:
https://spb.hh.ru/vacancy/3

обязанности:
https://spb.hh.ru/vacancy/35654309
https://spb.hh.ru/vacancy/36497374
обязанности:
https://spb.hh.ru/vacancy/36263834
обязанности:
https://spb.hh.ru/vacancy/36313589
https://spb.hh.ru/vacancy/36266440
https://spb.hh.ru/vacancy/36329488
обязанности:
https://spb.hh.ru/vacancy/36312565
https://spb.hh.ru/vacancy/35455181
обязанности:
https://spb.hh.ru/vacancy/36166312
https://spb.hh.ru/vacancy/20221152
обязанности:
https://spb.hh.ru/vacancy/36228359
обязанности:
https://spb.hh.ru/vacancy/36364907
https://spb.hh.ru/vacancy/33049315
обязанности:
https://spb.hh.ru/vacancy/36242204
должностные обязанности:
https://spb.hh.ru/vacancy/35152169
обязанности:
https://spb.hh.ru/vacancy/36354304
https://spb.hh.ru/vacancy/36099197
обязанности:
https://spb.hh.ru/vacancy/36474385
https://spb.hh.ru/vacancy/36189094
https://spb.hh.ru/vacancy/35510185
https://spb.hh.ru/vacancy/36081172
обязанности:
https://spb.hh.ru/vacancy/36247470
обязанности:
https://spb.hh.ru/vacancy/36319564
https://spb.hh

обязанности:
https://spb.hh.ru/vacancy/36427446
обязанности:
https://spb.hh.ru/vacancy/36346012
https://spb.hh.ru/vacancy/36114027
https://spb.hh.ru/vacancy/35471911
https://spb.hh.ru/vacancy/34951653
https://spb.hh.ru/vacancy/36496418
https://spb.hh.ru/vacancy/36443428
обязанности
https://spb.hh.ru/vacancy/36231119
https://spb.hh.ru/vacancy/36229084
https://spb.hh.ru/vacancy/36188834
обязанности:
https://spb.hh.ru/vacancy/36114426
https://spb.hh.ru/vacancy/36496627
https://spb.hh.ru/vacancy/36499143
обязанности:
https://spb.hh.ru/vacancy/36404834
обязанности:
https://spb.hh.ru/vacancy/36166797
https://spb.hh.ru/vacancy/36054711
https://spb.hh.ru/vacancy/36319406
обязанности:
https://spb.hh.ru/vacancy/36413914
https://spb.hh.ru/vacancy/36115846
обязанности:
https://spb.hh.ru/vacancy/36128054
обязанности:
https://spb.hh.ru/vacancy/36292910
обязанности:
https://spb.hh.ru/vacancy/36424424
должностные обязанности:
https://spb.hh.ru/vacancy/36332705
обязанности:
https://spb.hh.ru/vacancy/36

In [33]:
data  = get_vacancies(40, 'https://spb.hh.ru/catalog/Medicina-Farmacevtika')
data.to_csv(r'Z:\Collecting_data_openedu\12.csv')

https://spb.hh.ru/vacancy/36444660
обязанности:
https://spb.hh.ru/vacancy/36145041
https://spb.hh.ru/vacancy/36378179
обязанности:
https://spb.hh.ru/vacancy/36313987
обязанности:
https://spb.hh.ru/vacancy/36311956
https://spb.hh.ru/vacancy/36306720
обязанности:
https://spb.hh.ru/vacancy/36378184
обязанности:
https://spb.hh.ru/vacancy/36371177
обязанности:
https://spb.hh.ru/vacancy/34996786
https://spb.hh.ru/vacancy/36002228
обязанности:
https://spb.hh.ru/vacancy/36320271
обязанности:
https://spb.hh.ru/vacancy/36364415
обязанности:
https://spb.hh.ru/vacancy/36110055
https://spb.hh.ru/vacancy/36428132
обязанности:
https://spb.hh.ru/vacancy/35008052
обязанности:
https://spb.hh.ru/vacancy/33370019
https://spb.hh.ru/vacancy/36122823
обязанности:
https://spb.hh.ru/vacancy/35275964
обязанности:
https://spb.hh.ru/vacancy/36529006
https://spb.hh.ru/vacancy/36313749
обязанности:
https://spb.hh.ru/vacancy/36521416
обязанности:
https://spb.hh.ru/vacancy/35757921
https://spb.hh.ru/vacancy/36205098


https://spb.hh.ru/vacancy/36093474
обязанности:
https://spb.hh.ru/vacancy/36086534
обязанности: 
https://spb.hh.ru/vacancy/36442480
https://spb.hh.ru/vacancy/36424418
обязанности
https://spb.hh.ru/vacancy/36260989
обязанности:
https://spb.hh.ru/vacancy/36341508
обязанности:
https://spb.hh.ru/vacancy/36166791
обязанности:
https://spb.hh.ru/vacancy/36380880
обязанности:
https://spb.hh.ru/vacancy/36510223
обязанности:
https://spb.hh.ru/vacancy/36393982
обязанности:
https://spb.hh.ru/vacancy/36282697
https://spb.hh.ru/vacancy/36474632
https://spb.hh.ru/vacancy/36403755
обязанности:
https://spb.hh.ru/vacancy/35476613
обязанности:
https://spb.hh.ru/vacancy/36235379
обязанности:
https://spb.hh.ru/vacancy/36312302
https://spb.hh.ru/vacancy/22644786
обязанности:
https://spb.hh.ru/vacancy/36404498
обязанности:
https://spb.hh.ru/vacancy/36080076
обязанности:
https://spb.hh.ru/vacancy/36392604
обязанности:
https://spb.hh.ru/vacancy/36506404
обязанности:
https://spb.hh.ru/vacancy/36460787
https://s

обязанности:
https://spb.hh.ru/vacancy/36444806
обязанности:
https://spb.hh.ru/vacancy/36187439
обязанности:
https://spb.hh.ru/vacancy/36259453
обязанности:
https://spb.hh.ru/vacancy/36191798
https://spb.hh.ru/vacancy/36428176
обязанности:
https://spb.hh.ru/vacancy/36373162
обязанности:
https://spb.hh.ru/vacancy/36167766
обязанности:
https://spb.hh.ru/vacancy/35944943
обязанности:
https://spb.hh.ru/vacancy/36358911
обязанности:
https://spb.hh.ru/vacancy/34710814
обязанности:
https://spb.hh.ru/vacancy/35850441
обязанности:
https://spb.hh.ru/vacancy/36182098
обязанности:
https://spb.hh.ru/vacancy/36253007
обязанности:
https://spb.hh.ru/vacancy/36476686
обязанности:
https://spb.hh.ru/vacancy/21979362
обязанности:
https://spb.hh.ru/vacancy/36145904
обязанности:
https://spb.hh.ru/vacancy/36306650
обязанности
https://spb.hh.ru/vacancy/36339502
обязанности:
https://spb.hh.ru/vacancy/36393885
https://spb.hh.ru/vacancy/36411021
обязанности:
https://spb.hh.ru/vacancy/36208993
обязанности:
https:

обязанности:
https://spb.hh.ru/vacancy/36392717
https://spb.hh.ru/vacancy/36110432
обязанности:
https://spb.hh.ru/vacancy/35189791
https://spb.hh.ru/vacancy/36234731
обязанности:
https://spb.hh.ru/vacancy/36414141
обязанности:
https://spb.hh.ru/vacancy/36456133
обязанности:
https://spb.hh.ru/vacancy/36153783
обязанности
https://spb.hh.ru/vacancy/35521316
обязанности:
https://spb.hh.ru/vacancy/36445307
обязанности
https://spb.hh.ru/vacancy/36073211
обязанности:
https://spb.hh.ru/vacancy/36275858
обязанности:
https://spb.hh.ru/vacancy/36132398
обязанности:
https://spb.hh.ru/vacancy/36313447
https://spb.hh.ru/vacancy/36204930
обязанности:
https://spb.hh.ru/vacancy/34993213
https://spb.hh.ru/vacancy/36199484
обязанности:
https://spb.hh.ru/vacancy/36273008
https://spb.hh.ru/vacancy/35792054
обязанности:
https://spb.hh.ru/vacancy/36234132
обязанности:
https://spb.hh.ru/vacancy/36336316
обязанности:
https://spb.hh.ru/vacancy/36214019
обязанности:
https://spb.hh.ru/vacancy/36527986
обязанности

обязанности:
https://spb.hh.ru/vacancy/34747348
https://spb.hh.ru/vacancy/35998244
обязанности:
https://spb.hh.ru/vacancy/36187418
обязанности:
https://spb.hh.ru/vacancy/36460409
обязанности:
https://spb.hh.ru/vacancy/36002290
https://spb.hh.ru/vacancy/36092290
обязанности:
https://spb.hh.ru/vacancy/35302522
обязанности:
https://spb.hh.ru/vacancy/35809477
https://spb.hh.ru/vacancy/35703578
обязанности
https://spb.hh.ru/vacancy/36229418
обязанности:
https://spb.hh.ru/vacancy/36167979
обязанности:
https://spb.hh.ru/vacancy/36187504
обязанности:
https://spb.hh.ru/vacancy/30540110
обязанности:
https://spb.hh.ru/vacancy/36020278
обязанности:
https://spb.hh.ru/vacancy/35942185
обязанности:
https://spb.hh.ru/vacancy/36400408
обязанности:
https://spb.hh.ru/vacancy/36174681
https://spb.hh.ru/vacancy/36335439
обязанности:
https://spb.hh.ru/vacancy/36002769
https://spb.hh.ru/vacancy/36213715
обязанности
https://spb.hh.ru/vacancy/36240209
https://spb.hh.ru/vacancy/36217225
обязанности:
https://spb

...

обязанности:
https://spb.hh.ru/vacancy/36470738
обязанности:
https://spb.hh.ru/vacancy/36256808
https://spb.hh.ru/vacancy/36408550
обязанности:
https://spb.hh.ru/vacancy/36311913
обязанности:
https://spb.hh.ru/vacancy/36157121
обязанности
https://spb.hh.ru/vacancy/27765200
обязанности:
https://spb.hh.ru/vacancy/35299468
обязанности:
https://spb.hh.ru/vacancy/35357201
обязанности:
https://spb.hh.ru/vacancy/36523709
https://spb.hh.ru/vacancy/35872310
https://spb.hh.ru/vacancy/33481668
обязанности:
https://spb.hh.ru/vacancy/35329259
обязанности:
https://spb.hh.ru/vacancy/26593229
ваши обязанности:
https://spb.hh.ru/vacancy/36096317
https://spb.hh.ru/vacancy/36421712
https://spb.hh.ru/vacancy/34847282
обязанности:
https://spb.hh.ru/vacancy/36106049
обязанности:
https://spb.hh.ru/vacancy/36178528
обязанности:
https://spb.hh.ru/vacancy/35502678
обязанности:
https://spb.hh.ru/vacancy/35673602
https://spb.hh.ru/vacancy/35939775
обязанности:
https://spb.hh.ru/vacancy/35704944
https://spb.hh.ru/

обязанности:
https://spb.hh.ru/vacancy/36181679
обязанности:
https://spb.hh.ru/vacancy/35915067
обязанности:
https://spb.hh.ru/vacancy/34664584
в ваши обязанности входит:
https://spb.hh.ru/vacancy/36375678
https://spb.hh.ru/vacancy/33925043
обязанности:
https://spb.hh.ru/vacancy/36163812
https://spb.hh.ru/vacancy/36523944
обязанности:
https://spb.hh.ru/vacancy/36108690
обязанности:
https://spb.hh.ru/vacancy/36140514
обязанности:
https://spb.hh.ru/vacancy/36356543
обязанности:
https://spb.hh.ru/vacancy/34546260
обязанности:
https://spb.hh.ru/vacancy/36400435
обязанности
https://spb.hh.ru/vacancy/35805892
обязанности:
https://spb.hh.ru/vacancy/33840012
обязанности:
https://spb.hh.ru/vacancy/36380982
обязанности:
https://spb.hh.ru/vacancy/36502757
обязанности:
https://spb.hh.ru/vacancy/34698167
обязанности:
https://spb.hh.ru/vacancy/34847997
обязанности:
https://spb.hh.ru/vacancy/34661076
обязанности:
https://spb.hh.ru/vacancy/36503926
https://spb.hh.ru/vacancy/36087619
обязанности:
https

обязанности:
https://spb.hh.ru/vacancy/36195200
обязанности:
https://spb.hh.ru/vacancy/34164481
обязанности:
https://spb.hh.ru/vacancy/35772787
обязанности:
https://spb.hh.ru/vacancy/35950017
https://spb.hh.ru/vacancy/36031672
обязанности:
https://spb.hh.ru/vacancy/36236897
обязанности:
https://spb.hh.ru/vacancy/36324388
https://spb.hh.ru/vacancy/36436193
обязанности:
https://spb.hh.ru/vacancy/33141333
обязанности:
https://spb.hh.ru/vacancy/36509539
обязанности:
https://spb.hh.ru/vacancy/36366588
обязанности:
https://spb.hh.ru/vacancy/36231491
https://spb.hh.ru/vacancy/35811247
основные обязанности, функции:
https://spb.hh.ru/vacancy/36110980
https://spb.hh.ru/vacancy/35648635
https://spb.hh.ru/vacancy/36165761
обязанности:
https://spb.hh.ru/vacancy/35290351
обязанности:
https://spb.hh.ru/vacancy/36140140
обязанности:
https://spb.hh.ru/vacancy/32260598
https://spb.hh.ru/vacancy/36090569
обязанности:
https://spb.hh.ru/vacancy/35560636
обязанности:
https://spb.hh.ru/vacancy/35681728
осно

обязанности:
https://spb.hh.ru/vacancy/35591649
обязанности:
https://spb.hh.ru/vacancy/35805690
обязанности
https://spb.hh.ru/vacancy/33141324
обязанности:
https://spb.hh.ru/vacancy/36514178
должностные обязанности:
https://spb.hh.ru/vacancy/36414062
обязанности:
https://spb.hh.ru/vacancy/36427360
https://spb.hh.ru/vacancy/36424589
обязанности:
https://spb.hh.ru/vacancy/36213032
обязанности:
https://spb.hh.ru/vacancy/35315054
обязанности: 
https://spb.hh.ru/vacancy/34940341
обязанности:
https://spb.hh.ru/vacancy/36213995
https://spb.hh.ru/vacancy/31327782
обязанности:
https://spb.hh.ru/vacancy/36212950
обязанности:
https://spb.hh.ru/vacancy/34419146
обязанности:
https://spb.hh.ru/vacancy/36505388
должностные обязанности:
https://spb.hh.ru/vacancy/34164741
обязанности:
https://spb.hh.ru/vacancy/35773058
обязанности:
https://spb.hh.ru/vacancy/35281392
https://spb.hh.ru/vacancy/34123870
https://spb.hh.ru/vacancy/36199581
обязанности:
https://spb.hh.ru/vacancy/36511738
https://spb.hh.ru/va

https://spb.hh.ru/vacancy/35514803
обязанности:
https://spb.hh.ru/vacancy/35142168
обязанности:
https://spb.hh.ru/vacancy/36269009
обязанности:
https://spb.hh.ru/vacancy/36283169
обязанности:
https://spb.hh.ru/vacancy/36441953
обязанности:
https://spb.hh.ru/vacancy/35058923
https://spb.hh.ru/vacancy/36369605
https://spb.hh.ru/vacancy/36199661
обязанности:
https://spb.hh.ru/vacancy/36131546
обязанности:
https://spb.hh.ru/vacancy/35242404
обязанности:
https://spb.hh.ru/vacancy/36524113
https://spb.hh.ru/vacancy/36499624
обязанности:
https://spb.hh.ru/vacancy/36518565
обязанности:
https://spb.hh.ru/vacancy/36155528
обязанности:
https://spb.hh.ru/vacancy/34098975
https://spb.hh.ru/vacancy/34732122
обязанности:
https://spb.hh.ru/vacancy/35853705
обязанности:
https://spb.hh.ru/vacancy/36400447
обязанности:
https://spb.hh.ru/vacancy/36444814
https://spb.hh.ru/vacancy/30831874
обязанности:
https://spb.hh.ru/vacancy/36459765
обязанности:
https://spb.hh.ru/vacancy/35258966
обязанности: 
https://

In [34]:
data  = get_vacancies(18, 'https://spb.hh.ru/catalog/Nauka-Obrazovanie')
data.to_csv(r'Z:\Collecting_data_openedu\13.csv')

https://spb.hh.ru/vacancy/36449026
обязанности:
https://spb.hh.ru/vacancy/36195921
обязанности:
https://spb.hh.ru/vacancy/36199801
https://spb.hh.ru/vacancy/36318010
обязанности:
https://spb.hh.ru/vacancy/36263960
обязанности:
https://spb.hh.ru/vacancy/36378184
https://spb.hh.ru/vacancy/36357962
https://spb.hh.ru/vacancy/36494464
обязанности:
https://spb.hh.ru/vacancy/36276518
обязанности
https://spb.hh.ru/vacancy/36316710
https://spb.hh.ru/vacancy/36432627
https://spb.hh.ru/vacancy/34446557
обязанности:
https://spb.hh.ru/vacancy/36117736
https://spb.hh.ru/vacancy/36432633
https://spb.hh.ru/vacancy/36432463
обязанности:
https://spb.hh.ru/vacancy/36437054
https://spb.hh.ru/vacancy/35816893
https://spb.hh.ru/vacancy/36375091
обязанности:
https://spb.hh.ru/vacancy/36529247
https://spb.hh.ru/vacancy/36432454
обязанности:
https://spb.hh.ru/vacancy/36282891
https://spb.hh.ru/vacancy/36293626
https://spb.hh.ru/vacancy/36432635
https://spb.hh.ru/vacancy/35619162
https://spb.hh.ru/vacancy/36356

обязанности:
https://spb.hh.ru/vacancy/36214553
обязанности:
https://spb.hh.ru/vacancy/35838041
обязанности:
https://spb.hh.ru/vacancy/36245080
обязанности: 
https://spb.hh.ru/vacancy/33026886
https://spb.hh.ru/vacancy/36395316
основные обязанности:
https://spb.hh.ru/vacancy/35036957
https://spb.hh.ru/vacancy/36395901
https://spb.hh.ru/vacancy/34728786
https://spb.hh.ru/vacancy/35937769
https://spb.hh.ru/vacancy/35937786
обязанности:
https://spb.hh.ru/vacancy/34556956
https://spb.hh.ru/vacancy/30842035
https://spb.hh.ru/vacancy/36428093
https://spb.hh.ru/vacancy/36375744
https://spb.hh.ru/vacancy/34075139
обязанности:
https://spb.hh.ru/vacancy/36413034
https://spb.hh.ru/vacancy/36376768
https://spb.hh.ru/vacancy/31990257
обязанности
https://spb.hh.ru/vacancy/36083618
обязанности:
https://spb.hh.ru/vacancy/34766942
https://spb.hh.ru/vacancy/36373416
https://spb.hh.ru/vacancy/36074538
https://spb.hh.ru/vacancy/36369359
https://spb.hh.ru/vacancy/36068717
https://spb.hh.ru/vacancy/36329467

обязанности:
https://spb.hh.ru/vacancy/36356972
обязанности:
https://spb.hh.ru/vacancy/36245029
https://spb.hh.ru/vacancy/36527168
обязанности:
https://spb.hh.ru/vacancy/36203633
обязанности:
https://spb.hh.ru/vacancy/36106740
обязанности:
https://spb.hh.ru/vacancy/36359813
https://spb.hh.ru/vacancy/36213189
https://spb.hh.ru/vacancy/36320666
обязанности:
https://spb.hh.ru/vacancy/36297100
должностные обязанности учителя начальных классов, четкое выполнение инструкций учителя начальных классов 
https://spb.hh.ru/vacancy/36477868
обязанности:
https://spb.hh.ru/vacancy/36501819
обязанности:
https://spb.hh.ru/vacancy/36495826
обязанности:
https://spb.hh.ru/vacancy/36282679
https://spb.hh.ru/vacancy/36182175
обязанности:
https://spb.hh.ru/vacancy/36087642
обязанности:
https://spb.hh.ru/vacancy/36187716
обязанности:
https://spb.hh.ru/vacancy/36271400
https://spb.hh.ru/vacancy/32393312
обязанности:
https://spb.hh.ru/vacancy/36229146
https://spb.hh.ru/vacancy/36182170
обязанности:
https://spb

обязанности:
https://spb.hh.ru/vacancy/36167256
обязанности:
https://spb.hh.ru/vacancy/36140140
обязанности:
https://spb.hh.ru/vacancy/35767027
обязанности:
https://spb.hh.ru/vacancy/35553777
обязанности:
https://spb.hh.ru/vacancy/35751570
https://spb.hh.ru/vacancy/36260514
обязанности:
https://spb.hh.ru/vacancy/34656580
https://spb.hh.ru/vacancy/35667495
обязанности:
https://spb.hh.ru/vacancy/36493960
https://spb.hh.ru/vacancy/34078054
https://spb.hh.ru/vacancy/34702790
обязанности
https://spb.hh.ru/vacancy/35829465
профессиональные обязанности:
https://spb.hh.ru/vacancy/36070409
обязанности:
https://spb.hh.ru/vacancy/36268817
обязанности:
https://spb.hh.ru/vacancy/35767027
обязанности:
https://spb.hh.ru/vacancy/35553777
обязанности:
https://spb.hh.ru/vacancy/35751570
https://spb.hh.ru/vacancy/36260514
обязанности:
https://spb.hh.ru/vacancy/34656580
https://spb.hh.ru/vacancy/35667495
https://spb.hh.ru/vacancy/34078054
https://spb.hh.ru/vacancy/34702790
обязанности
https://spb.hh.ru/va

обязанности:
https://spb.hh.ru/vacancy/36351655
обязанности:
https://spb.hh.ru/vacancy/36386007
обязанности:
https://spb.hh.ru/vacancy/36386306
обязанности
https://spb.hh.ru/vacancy/33040855
обязанности:
https://spb.hh.ru/vacancy/34259639
обязанности:
https://spb.hh.ru/vacancy/35427020
https://spb.hh.ru/vacancy/36168965
обязанности:
https://spb.hh.ru/vacancy/36331039
обязанности:
https://spb.hh.ru/vacancy/36058051
должностные обязанности воспитателя , четкое выполнение инструкций воспитателя 
https://spb.hh.ru/vacancy/36091594
обязанности:
https://spb.hh.ru/vacancy/36119870
обязанности:
https://spb.hh.ru/vacancy/36120927
обязанности:
https://spb.hh.ru/vacancy/36186171
обязанности:
https://spb.hh.ru/vacancy/36305774
обязанности:
https://spb.hh.ru/vacancy/36370090
обязанности:
https://spb.hh.ru/vacancy/36370247
обязанности:
https://spb.hh.ru/vacancy/36370380
https://spb.hh.ru/vacancy/35793100
обязанности
https://spb.hh.ru/vacancy/35831451
обязанности:
https://spb.hh.ru/vacancy/36356631
о

In [35]:
data  = get_vacancies(40, 'https://spb.hh.ru/catalog/Nachalo-karery-Studenty')
data.to_csv(r'Z:\Collecting_data_openedu\14.csv')

https://spb.hh.ru/vacancy/31917545
https://spb.hh.ru/vacancy/35880324
https://spb.hh.ru/vacancy/17503202
https://spb.hh.ru/vacancy/36403144
обязанности:
https://spb.hh.ru/vacancy/36217443
https://spb.hh.ru/vacancy/36446388
https://spb.hh.ru/vacancy/33388007
обязанности:
https://spb.hh.ru/vacancy/36397226
обязанности:
https://spb.hh.ru/vacancy/36217465
https://spb.hh.ru/vacancy/36217357
https://spb.hh.ru/vacancy/36293071
обязанности:
https://spb.hh.ru/vacancy/36397224
обязанности:
https://spb.hh.ru/vacancy/36449287
обязанности:
https://spb.hh.ru/vacancy/35674791
https://spb.hh.ru/vacancy/36182713
https://spb.hh.ru/vacancy/35076726
https://spb.hh.ru/vacancy/34786132
https://spb.hh.ru/vacancy/36283001
https://spb.hh.ru/vacancy/36523823
https://spb.hh.ru/vacancy/36376437
https://spb.hh.ru/vacancy/35554886
https://spb.hh.ru/vacancy/36527947
обязанности:
https://spb.hh.ru/vacancy/36205406
https://spb.hh.ru/vacancy/36145915
https://spb.hh.ru/vacancy/36448957
https://spb.hh.ru/vacancy/35821797

https://spb.hh.ru/vacancy/36241291
https://spb.hh.ru/vacancy/36157890
https://spb.hh.ru/vacancy/36369413
ваши будущие обязанности:
https://spb.hh.ru/vacancy/36062423
https://spb.hh.ru/vacancy/36169956
обязанности:
https://spb.hh.ru/vacancy/36472288
https://spb.hh.ru/vacancy/33178095
https://spb.hh.ru/vacancy/32498714
https://spb.hh.ru/vacancy/36108940
https://spb.hh.ru/vacancy/36412310
обязанности:
https://spb.hh.ru/vacancy/36427313
обязанности:
https://spb.hh.ru/vacancy/33860847
обязанности:
https://spb.hh.ru/vacancy/36422011
обязанности:
https://spb.hh.ru/vacancy/35836991
обязанности:
https://spb.hh.ru/vacancy/35751115
https://spb.hh.ru/vacancy/36260903
https://spb.hh.ru/vacancy/23988035
https://spb.hh.ru/vacancy/35956709
https://spb.hh.ru/vacancy/36108241
обязанности:
https://spb.hh.ru/vacancy/36512194
https://spb.hh.ru/vacancy/36262981
должностные обязанности:
https://spb.hh.ru/vacancy/36410695
обязанности:
https://spb.hh.ru/vacancy/27854666
https://spb.hh.ru/vacancy/36358573
обяза

https://spb.hh.ru/vacancy/36090386
https://spb.hh.ru/vacancy/36446826
обязанности:
https://spb.hh.ru/vacancy/36466149
в ваши обязанности будет входить:
https://spb.hh.ru/vacancy/36197550
https://spb.hh.ru/vacancy/36450145
https://spb.hh.ru/vacancy/35934472
https://spb.hh.ru/vacancy/36443410
https://spb.hh.ru/vacancy/36458852
обязанности:
https://spb.hh.ru/vacancy/36445692
https://spb.hh.ru/vacancy/36428713
обязанности:
https://spb.hh.ru/vacancy/36463053
в ваши обязанности будет входить:
https://spb.hh.ru/vacancy/36189699
обязанности:
https://spb.hh.ru/vacancy/35558819
https://spb.hh.ru/vacancy/36469591
обязанности:
https://spb.hh.ru/vacancy/36286019
в ваши обязанности будет входить:
https://spb.hh.ru/vacancy/36189695
обязанности:
https://spb.hh.ru/vacancy/31980282
обязанности:
https://spb.hh.ru/vacancy/36419504
обязанности:
https://spb.hh.ru/vacancy/36466489
обязанности:
https://spb.hh.ru/vacancy/36176566
обязанности:
https://spb.hh.ru/vacancy/36346413
обязанности:
https://spb.hh.ru/va

обязанности:
https://spb.hh.ru/vacancy/34353889
обязанности:
https://spb.hh.ru/vacancy/36020339
обязанности:
https://spb.hh.ru/vacancy/36259462
https://spb.hh.ru/vacancy/36359558
обязанности:
https://spb.hh.ru/vacancy/31673298
наши обязанности:
https://spb.hh.ru/vacancy/35352625
https://spb.hh.ru/vacancy/35781070
https://spb.hh.ru/vacancy/35865991
обязанности: 
https://spb.hh.ru/vacancy/36110475
обязанности:
https://spb.hh.ru/vacancy/36145765
https://spb.hh.ru/vacancy/36404175
https://spb.hh.ru/vacancy/36405870
обязанности:
https://spb.hh.ru/vacancy/36053812
https://spb.hh.ru/vacancy/36224327
https://spb.hh.ru/vacancy/35606757
обязанности:
https://spb.hh.ru/vacancy/34939362
обязанности:
https://spb.hh.ru/vacancy/36130684
обязанности:
https://spb.hh.ru/vacancy/26080481
https://spb.hh.ru/vacancy/36085977
https://spb.hh.ru/vacancy/34708141
обязанности
https://spb.hh.ru/vacancy/35380634
обязанности:
https://spb.hh.ru/vacancy/36336180
обязанности:
https://spb.hh.ru/vacancy/36053966
обязанно

обязанности:
https://spb.hh.ru/vacancy/36173907
https://spb.hh.ru/vacancy/36196325
обязанности:
https://spb.hh.ru/vacancy/36291691
https://spb.hh.ru/vacancy/36297265
обязанности:
https://spb.hh.ru/vacancy/36279181
https://spb.hh.ru/vacancy/29224467
https://spb.hh.ru/vacancy/36263560
https://spb.hh.ru/vacancy/35459902
https://spb.hh.ru/vacancy/36292771
https://spb.hh.ru/vacancy/36215981
обязанности: 
https://spb.hh.ru/vacancy/36109843
обязанности:
https://spb.hh.ru/vacancy/29224155
https://spb.hh.ru/vacancy/36262574
https://spb.hh.ru/vacancy/35856912
обязанности:
https://spb.hh.ru/vacancy/35491650
должностные обязанности:
https://spb.hh.ru/vacancy/34998328
https://spb.hh.ru/vacancy/36295136
обязанности:
https://spb.hh.ru/vacancy/36261904
обязанности:
https://spb.hh.ru/vacancy/31387022
https://spb.hh.ru/vacancy/36295144
https://spb.hh.ru/vacancy/36282133
https://spb.hh.ru/vacancy/35485331
https://spb.hh.ru/vacancy/34223105
обязанности:
https://spb.hh.ru/vacancy/36359312
https://spb.hh.ru

...

обязанности:
https://spb.hh.ru/vacancy/35177216
обязанности:
https://spb.hh.ru/vacancy/35328182
https://spb.hh.ru/vacancy/36228958
https://spb.hh.ru/vacancy/36499731
https://spb.hh.ru/vacancy/33888040
обязанности:
https://spb.hh.ru/vacancy/36528483
https://spb.hh.ru/vacancy/35996365
обязанности:
https://spb.hh.ru/vacancy/36338215
https://spb.hh.ru/vacancy/35081396
обязанности:
https://spb.hh.ru/vacancy/36065407
https://spb.hh.ru/vacancy/36405870
https://spb.hh.ru/vacancy/36315992
https://spb.hh.ru/vacancy/36358300
обязанности:
https://spb.hh.ru/vacancy/36427628
https://spb.hh.ru/vacancy/36161162
https://spb.hh.ru/vacancy/36422467
https://spb.hh.ru/vacancy/36330632
https://spb.hh.ru/vacancy/36463862
обязанности
https://spb.hh.ru/vacancy/36370138
обязанности:
https://spb.hh.ru/vacancy/36202716
обязанности:
https://spb.hh.ru/vacancy/36110625
обязанности:
https://spb.hh.ru/vacancy/35611454
обязанности:
https://spb.hh.ru/vacancy/36120276
https://spb.hh.ru/vacancy/36525562
обязанности:
https

https://spb.hh.ru/vacancy/36203157
обязанности:
https://spb.hh.ru/vacancy/36370197
обязанности:
https://spb.hh.ru/vacancy/35673152
обязанности:
https://spb.hh.ru/vacancy/36156174
обязанности:
https://spb.hh.ru/vacancy/35162123
обязанности:
https://spb.hh.ru/vacancy/36443569
https://spb.hh.ru/vacancy/32428619
https://spb.hh.ru/vacancy/36278214
обязанности:
https://spb.hh.ru/vacancy/36144480
https://spb.hh.ru/vacancy/36182646
https://spb.hh.ru/vacancy/35772554
https://spb.hh.ru/vacancy/36210676
обязанности:
https://spb.hh.ru/vacancy/36186408
https://spb.hh.ru/vacancy/36316225
обязанности:
https://spb.hh.ru/vacancy/36182496
обязанности:
https://spb.hh.ru/vacancy/36242110
обязанности:
https://spb.hh.ru/vacancy/36162321
https://spb.hh.ru/vacancy/36493851
обязанности:
https://spb.hh.ru/vacancy/36235786
https://spb.hh.ru/vacancy/35058040
обязанности:
https://spb.hh.ru/vacancy/36278334
должностные обязанности:
https://spb.hh.ru/vacancy/36302147
https://spb.hh.ru/vacancy/36329928
обязанности: 


https://spb.hh.ru/vacancy/36402572
https://spb.hh.ru/vacancy/36134216
https://spb.hh.ru/vacancy/34210847
https://spb.hh.ru/vacancy/36104302
обязанности:
https://spb.hh.ru/vacancy/35609793
обязанности:
https://spb.hh.ru/vacancy/35373574
https://spb.hh.ru/vacancy/36326841
обязанности:
https://spb.hh.ru/vacancy/36307430
https://spb.hh.ru/vacancy/36523785
обязанности:
https://spb.hh.ru/vacancy/36319672
https://spb.hh.ru/vacancy/25519241
https://spb.hh.ru/vacancy/36208058
https://spb.hh.ru/vacancy/35432219
обязанности:
https://spb.hh.ru/vacancy/36109381
должностные обязанности:
https://spb.hh.ru/vacancy/36238588
https://spb.hh.ru/vacancy/35322603
обязанности:
https://spb.hh.ru/vacancy/34609536
обязанности:
https://spb.hh.ru/vacancy/34827588
https://spb.hh.ru/vacancy/36335550
обязанности:
https://spb.hh.ru/vacancy/35449062
https://spb.hh.ru/vacancy/34199344
https://spb.hh.ru/vacancy/36266620
https://spb.hh.ru/vacancy/35869320
https://spb.hh.ru/vacancy/36422508
https://spb.hh.ru/vacancy/36374

https://spb.hh.ru/vacancy/32793492
https://spb.hh.ru/vacancy/32803977
обязанности
https://spb.hh.ru/vacancy/36011383
https://spb.hh.ru/vacancy/36388565
https://spb.hh.ru/vacancy/36502994
обязанности:
https://spb.hh.ru/vacancy/36308630
https://spb.hh.ru/vacancy/36249292
https://spb.hh.ru/vacancy/34245820
https://spb.hh.ru/vacancy/25022426
https://spb.hh.ru/vacancy/26809085
https://spb.hh.ru/vacancy/36165282
https://spb.hh.ru/vacancy/36011253
https://spb.hh.ru/vacancy/36118897
обязанности:
https://spb.hh.ru/vacancy/35331101
обязанности:
https://spb.hh.ru/vacancy/36463485
обязанности:
https://spb.hh.ru/vacancy/36410918
https://spb.hh.ru/vacancy/33349281
https://spb.hh.ru/vacancy/36465313
https://spb.hh.ru/vacancy/35295370
https://spb.hh.ru/vacancy/36185683
обязанности:
https://spb.hh.ru/vacancy/34190077
обязанности:
https://spb.hh.ru/vacancy/35800293
обязанности:
https://spb.hh.ru/vacancy/29090621
https://spb.hh.ru/vacancy/36452360
https://spb.hh.ru/vacancy/34089874
обязанности:
https://s

https://spb.hh.ru/vacancy/30844452
https://spb.hh.ru/vacancy/36368228
https://spb.hh.ru/vacancy/36503900
https://spb.hh.ru/vacancy/34386679
https://spb.hh.ru/vacancy/36214480
обязанности:
https://spb.hh.ru/vacancy/36185734
https://spb.hh.ru/vacancy/36208256
https://spb.hh.ru/vacancy/36113912
https://spb.hh.ru/vacancy/36190512
https://spb.hh.ru/vacancy/34993958
https://spb.hh.ru/vacancy/36202733
https://spb.hh.ru/vacancy/32558075
https://spb.hh.ru/vacancy/33526462
https://spb.hh.ru/vacancy/36518801
https://spb.hh.ru/vacancy/35182722
https://spb.hh.ru/vacancy/34442895
https://spb.hh.ru/vacancy/35514840
https://spb.hh.ru/vacancy/36206907
обязанности:
https://spb.hh.ru/vacancy/32284892
https://spb.hh.ru/vacancy/36251904
https://spb.hh.ru/vacancy/36197867
https://spb.hh.ru/vacancy/36053356
https://spb.hh.ru/vacancy/36138929


In [36]:
data  = get_vacancies(40, 'https://spb.hh.ru/catalog/Prodazhi')
data.to_csv(r'Z:\Collecting_data_openedu\15.csv')

https://spb.hh.ru/vacancy/33269131
https://spb.hh.ru/vacancy/31917545
https://spb.hh.ru/vacancy/35984809
https://spb.hh.ru/vacancy/35770268
обязанности: 
https://spb.hh.ru/vacancy/36271679
в твои обязанности будет входить:
https://spb.hh.ru/vacancy/28390409
https://spb.hh.ru/vacancy/34747825
https://spb.hh.ru/vacancy/35880324
обязанности:
https://spb.hh.ru/vacancy/35730216
https://spb.hh.ru/vacancy/36088916
https://spb.hh.ru/vacancy/36512510
https://spb.hh.ru/vacancy/36118137
https://spb.hh.ru/vacancy/17503202
обязанности:
https://spb.hh.ru/vacancy/36032761
https://spb.hh.ru/vacancy/36493492
https://spb.hh.ru/vacancy/35262030
https://spb.hh.ru/vacancy/36403144
https://spb.hh.ru/vacancy/34577987
https://spb.hh.ru/vacancy/36464114
https://spb.hh.ru/vacancy/36449841
https://spb.hh.ru/vacancy/35213439
https://spb.hh.ru/vacancy/26216374
https://spb.hh.ru/vacancy/28044821
https://spb.hh.ru/vacancy/35209564
https://spb.hh.ru/vacancy/31672284
обязанности:
https://spb.hh.ru/vacancy/36150609
htt

обязанности:
https://spb.hh.ru/vacancy/36214143
профессиональные требования и обязанности:
https://spb.hh.ru/vacancy/36215850
https://spb.hh.ru/vacancy/36372539
https://spb.hh.ru/vacancy/36287309
https://spb.hh.ru/vacancy/33177919
https://spb.hh.ru/vacancy/36527687
обязанности:
https://spb.hh.ru/vacancy/36209453
обязанности:
https://spb.hh.ru/vacancy/36276082
https://spb.hh.ru/vacancy/36205494
https://spb.hh.ru/vacancy/35272714
обязанности:
https://spb.hh.ru/vacancy/35009384
https://spb.hh.ru/vacancy/36136790
обязанности:
https://spb.hh.ru/vacancy/35680008
обязанности:
https://spb.hh.ru/vacancy/36206312
https://spb.hh.ru/vacancy/36207710
обязанности:
https://spb.hh.ru/vacancy/36494726
https://spb.hh.ru/vacancy/36304302
https://spb.hh.ru/vacancy/35444033
обязанности:
https://spb.hh.ru/vacancy/36299411
обязанности:
https://spb.hh.ru/vacancy/36397247
https://spb.hh.ru/vacancy/36445858
https://spb.hh.ru/vacancy/36337477
обязанности:
https://spb.hh.ru/vacancy/35704103
обязанности:
https://s

https://spb.hh.ru/vacancy/36518370
обязанности:
https://spb.hh.ru/vacancy/36143638
https://spb.hh.ru/vacancy/35507231
https://spb.hh.ru/vacancy/36123233
наши обязанности: 
https://spb.hh.ru/vacancy/36282392
дистанционный менеджер по продажам ( договор гпх)обязанности:
https://spb.hh.ru/vacancy/36211662
должностные обязанности:
https://spb.hh.ru/vacancy/36435680
обязанности:
https://spb.hh.ru/vacancy/36204199
https://spb.hh.ru/vacancy/36288981
обязанности:
https://spb.hh.ru/vacancy/36212886
https://spb.hh.ru/vacancy/36287266
обязанности
https://spb.hh.ru/vacancy/36068214
https://spb.hh.ru/vacancy/36205284
обязанности:
https://spb.hh.ru/vacancy/36262956
https://spb.hh.ru/vacancy/35681285
https://spb.hh.ru/vacancy/36434812
https://spb.hh.ru/vacancy/36200336
обязанности:
https://spb.hh.ru/vacancy/35777011
обязанности:
https://spb.hh.ru/vacancy/35928924
https://spb.hh.ru/vacancy/36395983
обязанности:
https://spb.hh.ru/vacancy/36292786
https://spb.hh.ru/vacancy/31867075
обязанности: 
https:/

обязанности:
https://spb.hh.ru/vacancy/36265893
https://spb.hh.ru/vacancy/35061944
https://spb.hh.ru/vacancy/35695238
обязанности:
https://spb.hh.ru/vacancy/36441528
https://spb.hh.ru/vacancy/35780561
обязанности:
https://spb.hh.ru/vacancy/36129739
обязанности:
https://spb.hh.ru/vacancy/36258579
обязанности:
https://spb.hh.ru/vacancy/36426311
https://spb.hh.ru/vacancy/35706648
https://spb.hh.ru/vacancy/36002863
https://spb.hh.ru/vacancy/36124802
должностные обязанности:
https://spb.hh.ru/vacancy/36523918
https://spb.hh.ru/vacancy/36516921
https://spb.hh.ru/vacancy/30914934
обязанности:
https://spb.hh.ru/vacancy/36259631
https://spb.hh.ru/vacancy/33841185
https://spb.hh.ru/vacancy/36098390
https://spb.hh.ru/vacancy/35432055
обязанности:
https://spb.hh.ru/vacancy/36259863
обязанности:
https://spb.hh.ru/vacancy/36159889
https://spb.hh.ru/vacancy/34183320
https://spb.hh.ru/vacancy/34492516
https://spb.hh.ru/vacancy/36359075
https://spb.hh.ru/vacancy/35776147
https://spb.hh.ru/vacancy/36256

обязанности:
https://spb.hh.ru/vacancy/36268432
https://spb.hh.ru/vacancy/35913556
обязанности: 
https://spb.hh.ru/vacancy/36343095
https://spb.hh.ru/vacancy/35920009
обязанности:
https://spb.hh.ru/vacancy/35470283
обязанности:
https://spb.hh.ru/vacancy/36231723
https://spb.hh.ru/vacancy/35779546
https://spb.hh.ru/vacancy/36108940
обязанности:
https://spb.hh.ru/vacancy/35836991
обязанности:
https://spb.hh.ru/vacancy/35859974
https://spb.hh.ru/vacancy/36395291
https://spb.hh.ru/vacancy/35075721
https://spb.hh.ru/vacancy/36421232
обязанности:
https://spb.hh.ru/vacancy/36503959
обязанности:
https://spb.hh.ru/vacancy/36153069
обязанности:
https://spb.hh.ru/vacancy/36266386
https://spb.hh.ru/vacancy/36146420
обязанности:
https://spb.hh.ru/vacancy/36211570
обязанности:
https://spb.hh.ru/vacancy/31484120
обязанности:
https://spb.hh.ru/vacancy/36415013
https://spb.hh.ru/vacancy/36065569
https://spb.hh.ru/vacancy/36412310
https://spb.hh.ru/vacancy/36427846
https://spb.hh.ru/vacancy/36422263
обя

...

https://spb.hh.ru/vacancy/36238884
https://spb.hh.ru/vacancy/36454953
должностные обязанности:
https://spb.hh.ru/vacancy/36470669
обязанности: 
https://spb.hh.ru/vacancy/23042102
https://spb.hh.ru/vacancy/35404209
должностные обязанности:
https://spb.hh.ru/vacancy/36383419
обязанности:
https://spb.hh.ru/vacancy/36099092
https://spb.hh.ru/vacancy/36411195
https://spb.hh.ru/vacancy/36086429
обязанности:
https://spb.hh.ru/vacancy/36066134
обязанности:
https://spb.hh.ru/vacancy/36088095
обязанности
https://spb.hh.ru/vacancy/36240932
https://spb.hh.ru/vacancy/36167906
обязанности:
https://spb.hh.ru/vacancy/35771884
обязанности:
https://spb.hh.ru/vacancy/36388028
https://spb.hh.ru/vacancy/35845336
обязанности:
https://spb.hh.ru/vacancy/36057417
обязанности:
https://spb.hh.ru/vacancy/36336953
https://spb.hh.ru/vacancy/36242561
обязанности:
https://spb.hh.ru/vacancy/36056682
обязанности:
https://spb.hh.ru/vacancy/36078946
обязанности:
https://spb.hh.ru/vacancy/36089156
https://spb.hh.ru/vacanc

https://spb.hh.ru/vacancy/35865062
https://spb.hh.ru/vacancy/36093602
обязанности:
https://spb.hh.ru/vacancy/36400021
https://spb.hh.ru/vacancy/36406938
обязанности:
https://spb.hh.ru/vacancy/35145915
обязанности:
https://spb.hh.ru/vacancy/36159556
обязанности:
https://spb.hh.ru/vacancy/36158871
обязанности:
https://spb.hh.ru/vacancy/36091308
https://spb.hh.ru/vacancy/36240757
обязанности
https://spb.hh.ru/vacancy/36327719
обязанности:
https://spb.hh.ru/vacancy/36407707
обязанности:
https://spb.hh.ru/vacancy/36083262
https://spb.hh.ru/vacancy/36339860
https://spb.hh.ru/vacancy/36099213
обязанности:
https://spb.hh.ru/vacancy/36413745
обязанности:
https://spb.hh.ru/vacancy/35814172
https://spb.hh.ru/vacancy/36158353
обязанности:
https://spb.hh.ru/vacancy/36322456
обязанности:
https://spb.hh.ru/vacancy/36158085
обязанности:
https://spb.hh.ru/vacancy/36390425
обязанности:
https://spb.hh.ru/vacancy/36220996
обязанности:
https://spb.hh.ru/vacancy/35309195
обязанности:
https://spb.hh.ru/vacan

https://spb.hh.ru/vacancy/36262875
https://spb.hh.ru/vacancy/16555352
https://spb.hh.ru/vacancy/35071783
обязанности:
https://spb.hh.ru/vacancy/36143838
должностные обязанности:
https://spb.hh.ru/vacancy/36461773
обязанности:
https://spb.hh.ru/vacancy/34011368
обязанности директора:
https://spb.hh.ru/vacancy/34860824
https://spb.hh.ru/vacancy/36318707
обязанности:
https://spb.hh.ru/vacancy/36201970
https://spb.hh.ru/vacancy/33131414
обязанности:
https://spb.hh.ru/vacancy/36202543
https://spb.hh.ru/vacancy/36469591
обязанности:
https://spb.hh.ru/vacancy/36448771
обязанности:
https://spb.hh.ru/vacancy/36463852
обязанности:
https://spb.hh.ru/vacancy/36426915
в ваши обязанности будут входить:
https://spb.hh.ru/vacancy/36206869
обязанности:
https://spb.hh.ru/vacancy/36463608
https://spb.hh.ru/vacancy/34921475
обязанности:
https://spb.hh.ru/vacancy/36461541
https://spb.hh.ru/vacancy/36238917
https://spb.hh.ru/vacancy/34649151
обязанности:
https://spb.hh.ru/vacancy/36287564
обязанности
https:

обязанности:
https://spb.hh.ru/vacancy/36440773
обязанности:
https://spb.hh.ru/vacancy/36060502
в ваши обязанности будет входить:
https://spb.hh.ru/vacancy/36437070
обязанности:
https://spb.hh.ru/vacancy/36447244
обязанности:
https://spb.hh.ru/vacancy/34847997
https://spb.hh.ru/vacancy/36183595
обязанности:
https://spb.hh.ru/vacancy/36050537
обязанности:
https://spb.hh.ru/vacancy/36228346
обязанности:
https://spb.hh.ru/vacancy/36426560
обязанности:
https://spb.hh.ru/vacancy/36287089
обязанности:
https://spb.hh.ru/vacancy/36440741
обязанности:
https://spb.hh.ru/vacancy/36444539
ваши обязанности:
https://spb.hh.ru/vacancy/35670299
обязанности:
https://spb.hh.ru/vacancy/36115466
https://spb.hh.ru/vacancy/36229088
https://spb.hh.ru/vacancy/36440599
https://spb.hh.ru/vacancy/35619202
https://spb.hh.ru/vacancy/36443859
https://spb.hh.ru/vacancy/33407385
обязанности:
https://spb.hh.ru/vacancy/36432626
в твои обязанности будет входить:
https://spb.hh.ru/vacancy/36082416
https://spb.hh.ru/vacan

https://spb.hh.ru/vacancy/36358135
https://spb.hh.ru/vacancy/36416242
https://spb.hh.ru/vacancy/36297182
обязанности:
https://spb.hh.ru/vacancy/36270706
обязанности:
https://spb.hh.ru/vacancy/36056653
обязанности:
https://spb.hh.ru/vacancy/36421409
обязанности:
https://spb.hh.ru/vacancy/36072640
https://spb.hh.ru/vacancy/35581968
https://spb.hh.ru/vacancy/35854351
обязанности:
https://spb.hh.ru/vacancy/36335827
https://spb.hh.ru/vacancy/36198062
обязанности:
https://spb.hh.ru/vacancy/36426347
https://spb.hh.ru/vacancy/34407499
https://spb.hh.ru/vacancy/36311478
ваши обязанности:
https://spb.hh.ru/vacancy/36431566
обязанности:
https://spb.hh.ru/vacancy/35355952
обязанности:
https://spb.hh.ru/vacancy/36431526
https://spb.hh.ru/vacancy/36432072
обязанности:
https://spb.hh.ru/vacancy/36416018
обязанности:
https://spb.hh.ru/vacancy/36425422
https://spb.hh.ru/vacancy/33866001
обязанности:
https://spb.hh.ru/vacancy/36433224
https://spb.hh.ru/vacancy/35327241
https://spb.hh.ru/vacancy/36428176

In [37]:
data  = get_vacancies(6, 'https://spb.hh.ru/catalog/Strahovanie')
data.to_csv(r'Z:\Collecting_data_openedu\16.csv')

https://spb.hh.ru/vacancy/26216374
обязанности:
https://spb.hh.ru/vacancy/36217468
обязанности:
https://spb.hh.ru/vacancy/35713806
обязанности:
https://spb.hh.ru/vacancy/36376209
обязанности: 
https://spb.hh.ru/vacancy/36376601
обязанности
https://spb.hh.ru/vacancy/36073624
обязанности:
https://spb.hh.ru/vacancy/36208821
обязанности:
https://spb.hh.ru/vacancy/36299433
обязанности:
https://spb.hh.ru/vacancy/36066989
обязанности:
https://spb.hh.ru/vacancy/36445307
обязанности:
https://spb.hh.ru/vacancy/36244657
https://spb.hh.ru/vacancy/36367493
обязанности:
https://spb.hh.ru/vacancy/36109635
https://spb.hh.ru/vacancy/35219162
https://spb.hh.ru/vacancy/31867075
обязанности:
https://spb.hh.ru/vacancy/36299459
https://spb.hh.ru/vacancy/29875926
обязанности:
https://spb.hh.ru/vacancy/36167280
https://spb.hh.ru/vacancy/35924230
https://spb.hh.ru/vacancy/36143509
https://spb.hh.ru/vacancy/30894319
обязанности:
https://spb.hh.ru/vacancy/35925262
https://spb.hh.ru/vacancy/36073501
https://spb.h

обязанности:
https://spb.hh.ru/vacancy/35153076
обязанности:
https://spb.hh.ru/vacancy/35428455
обязанности:
https://spb.hh.ru/vacancy/35881377
обязанности:
https://spb.hh.ru/vacancy/35154128
должностные обязанности:
https://spb.hh.ru/vacancy/31410412
обязанности:
https://spb.hh.ru/vacancy/36217255
обязанности:
https://spb.hh.ru/vacancy/35626174
обязанности:
https://spb.hh.ru/vacancy/36331870
ваши обязанности:
https://spb.hh.ru/vacancy/36217157
должностные обязанности:
https://spb.hh.ru/vacancy/35767387
обязанности:
https://spb.hh.ru/vacancy/36287509
должностные обязанности:
https://spb.hh.ru/vacancy/36012884
основные обязанности:
https://spb.hh.ru/vacancy/34382315
https://spb.hh.ru/vacancy/35811280
https://spb.hh.ru/vacancy/36121332
обязанности:
https://spb.hh.ru/vacancy/36286813
обязанности:
https://spb.hh.ru/vacancy/36413961
обязанности:
https://spb.hh.ru/vacancy/36075165
обязанности:
https://spb.hh.ru/vacancy/36340149
обязанности:
https://spb.hh.ru/vacancy/36166602
https://spb.hh.r

In [38]:
data  = get_vacancies(40, 'https://spb.hh.ru/catalog/Stroitelstvo-Nedvizhimost')
data.to_csv(r'Z:\Collecting_data_openedu\17.csv')

обязанности:
https://spb.hh.ru/vacancy/36424862
обязанности:
https://spb.hh.ru/vacancy/36032761
https://spb.hh.ru/vacancy/35262030
https://spb.hh.ru/vacancy/28044821
https://spb.hh.ru/vacancy/36442346
https://spb.hh.ru/vacancy/31383178
https://spb.hh.ru/vacancy/35964024
https://spb.hh.ru/vacancy/35942415
https://spb.hh.ru/vacancy/36315664
https://spb.hh.ru/vacancy/36378218
https://spb.hh.ru/vacancy/36223236
https://spb.hh.ru/vacancy/36438659
обязанности:
https://spb.hh.ru/vacancy/34352709
обязанности:
https://spb.hh.ru/vacancy/36216064
обязанности:
https://spb.hh.ru/vacancy/36320515
https://spb.hh.ru/vacancy/36145102
обязанности:
https://spb.hh.ru/vacancy/36216267
обязанности:
https://spb.hh.ru/vacancy/36210502
обязанности:
https://spb.hh.ru/vacancy/36146017
обязанности:
https://spb.hh.ru/vacancy/36378181
https://spb.hh.ru/vacancy/36208721
https://spb.hh.ru/vacancy/33177716
обязанности:
https://spb.hh.ru/vacancy/36141882
обязанности
https://spb.hh.ru/vacancy/36140505
должностные обязан

обязанности:
https://spb.hh.ru/vacancy/36114232
обязанности:
https://spb.hh.ru/vacancy/35934015
обязанности:
https://spb.hh.ru/vacancy/36069086
обязанности:
https://spb.hh.ru/vacancy/36263352
обязанности:
https://spb.hh.ru/vacancy/36180640
обязанности:
https://spb.hh.ru/vacancy/36427478
обязанности:
https://spb.hh.ru/vacancy/36477284
обязанности:
https://spb.hh.ru/vacancy/35308961
обязанности:
https://spb.hh.ru/vacancy/36166287
https://spb.hh.ru/vacancy/36359931
обязанности:
https://spb.hh.ru/vacancy/36356074
https://spb.hh.ru/vacancy/36319679
обязанности:
https://spb.hh.ru/vacancy/36501773
https://spb.hh.ru/vacancy/36163034
обязанности:
https://spb.hh.ru/vacancy/36262625
https://spb.hh.ru/vacancy/36158789
обязанности:
https://spb.hh.ru/vacancy/36068930
обязанности:
https://spb.hh.ru/vacancy/36413193
обязанности:
https://spb.hh.ru/vacancy/33010300
обязанности:
https://spb.hh.ru/vacancy/36234952
обязанности:
https://spb.hh.ru/vacancy/36140703
https://spb.hh.ru/vacancy/36413986
https://s

обязанности:
https://spb.hh.ru/vacancy/36123652
обязанности:
https://spb.hh.ru/vacancy/36444322
обязанности:
https://spb.hh.ru/vacancy/36428376
обязанности:
https://spb.hh.ru/vacancy/34077318
https://spb.hh.ru/vacancy/35945299
https://spb.hh.ru/vacancy/36432413
обязанности:
https://spb.hh.ru/vacancy/36154162
должностные обязанности:
https://spb.hh.ru/vacancy/36435630
https://spb.hh.ru/vacancy/36431469
https://spb.hh.ru/vacancy/36116309
обязанности
https://spb.hh.ru/vacancy/36084902
https://spb.hh.ru/vacancy/36340079
обязанности:
https://spb.hh.ru/vacancy/35945010
обязанности:
https://spb.hh.ru/vacancy/35336161
обязанности:
https://spb.hh.ru/vacancy/36416263
обязанности:
https://spb.hh.ru/vacancy/36026483
обязанности:
https://spb.hh.ru/vacancy/36433093
https://spb.hh.ru/vacancy/36351665
https://spb.hh.ru/vacancy/36426677
обязанности:
https://spb.hh.ru/vacancy/36406765
обязанности:
https://spb.hh.ru/vacancy/36423003
обязанности:
https://spb.hh.ru/vacancy/36204022
обязанности:
https://spb

обязанности:
https://spb.hh.ru/vacancy/33581146
https://spb.hh.ru/vacancy/36203979
https://spb.hh.ru/vacancy/35336308
обязанности:
https://spb.hh.ru/vacancy/36200617
https://spb.hh.ru/vacancy/36262771
обязанности:
https://spb.hh.ru/vacancy/36070319
обязанности:
https://spb.hh.ru/vacancy/36244771
обязанности:
https://spb.hh.ru/vacancy/36078367
обязанности:
https://spb.hh.ru/vacancy/36197992
обязанности:
https://spb.hh.ru/vacancy/36233253
обязанности:
https://spb.hh.ru/vacancy/36231549
https://spb.hh.ru/vacancy/36218063
основные обязанности :
https://spb.hh.ru/vacancy/36228197
обязанности:
https://spb.hh.ru/vacancy/35851471
https://spb.hh.ru/vacancy/36224702
обязанности:
https://spb.hh.ru/vacancy/34764912
обязанности: 
https://spb.hh.ru/vacancy/36215919
обязанности:
https://spb.hh.ru/vacancy/35288508
обязанности: 
https://spb.hh.ru/vacancy/36330466
обязанности:
https://spb.hh.ru/vacancy/36346825
обязанности:
https://spb.hh.ru/vacancy/36195950
https://spb.hh.ru/vacancy/36190297
https://sp

обязанности:
https://spb.hh.ru/vacancy/35701443
https://spb.hh.ru/vacancy/36205078
обязанности:
https://spb.hh.ru/vacancy/35133741
обязанности:
https://spb.hh.ru/vacancy/36097399
обязанности:
https://spb.hh.ru/vacancy/36094669
обязанности
https://spb.hh.ru/vacancy/36077983
https://spb.hh.ru/vacancy/35631436
обязанности:
https://spb.hh.ru/vacancy/36121564
https://spb.hh.ru/vacancy/36289281
обязанности:
https://spb.hh.ru/vacancy/36101902
обязанности:
https://spb.hh.ru/vacancy/34670304
обязанности:
https://spb.hh.ru/vacancy/35692672
обязанности:
https://spb.hh.ru/vacancy/35127125
обязанности:
https://spb.hh.ru/vacancy/36090074
https://spb.hh.ru/vacancy/35196909
обязанности:
https://spb.hh.ru/vacancy/36086295
обязанности:
https://spb.hh.ru/vacancy/36093022
обязанности:
https://spb.hh.ru/vacancy/35493271
обязанности:
https://spb.hh.ru/vacancy/35902591
https://spb.hh.ru/vacancy/36087180
обязанности:
https://spb.hh.ru/vacancy/35197439
обязанности:
https://spb.hh.ru/vacancy/35743258
https://sp

...

обязанности:
https://spb.hh.ru/vacancy/36294885
обязанности:
https://spb.hh.ru/vacancy/36276906
обязанности:
https://spb.hh.ru/vacancy/36067055
обязанности:
https://spb.hh.ru/vacancy/36163958
обязанности:
https://spb.hh.ru/vacancy/36293381
обязанности:
https://spb.hh.ru/vacancy/36480061
в ваши обязанности будет входить:
https://spb.hh.ru/vacancy/36307643
обязанности:
https://spb.hh.ru/vacancy/36387265
обязанности:
https://spb.hh.ru/vacancy/32111239
обязанности:
https://spb.hh.ru/vacancy/35509924
https://spb.hh.ru/vacancy/36390376
обязанности:
https://spb.hh.ru/vacancy/36461949
https://spb.hh.ru/vacancy/36444712
обязанности:
https://spb.hh.ru/vacancy/36073768
обязанности:
https://spb.hh.ru/vacancy/36243048
обязанности:
https://spb.hh.ru/vacancy/36172411
обязанности:
https://spb.hh.ru/vacancy/36313604
обязанности:
https://spb.hh.ru/vacancy/36448427
обязанности:
https://spb.hh.ru/vacancy/35089151
обязанности:
https://spb.hh.ru/vacancy/36419032
https://spb.hh.ru/vacancy/36408866
обязанност

https://spb.hh.ru/vacancy/35882083
обязанности:
https://spb.hh.ru/vacancy/36244812
обязанности:
https://spb.hh.ru/vacancy/36317718
обязанности:
https://spb.hh.ru/vacancy/36402027
обязанности:
https://spb.hh.ru/vacancy/36089993
обязанности:
https://spb.hh.ru/vacancy/36506757
обязанности:
https://spb.hh.ru/vacancy/35501730
обязанности:
https://spb.hh.ru/vacancy/34984375
обязанности:
https://spb.hh.ru/vacancy/36302849
обязанности:
https://spb.hh.ru/vacancy/36332414
https://spb.hh.ru/vacancy/36374563
обязанности
https://spb.hh.ru/vacancy/36179817
обязанности:
https://spb.hh.ru/vacancy/36390118
должностные обязанности:
https://spb.hh.ru/vacancy/36206715
обязанности:
https://spb.hh.ru/vacancy/34729472
https://spb.hh.ru/vacancy/36438051
https://spb.hh.ru/vacancy/35623907
обязанности:
https://spb.hh.ru/vacancy/35937918
обязанности:
https://spb.hh.ru/vacancy/35730813
обязанности:
https://spb.hh.ru/vacancy/36327315
обязанности:
https://spb.hh.ru/vacancy/36422794
обязанности:
https://spb.hh.ru/va

обязанности:
https://spb.hh.ru/vacancy/36161807
обязанности:
https://spb.hh.ru/vacancy/36118935
обязанности:
https://spb.hh.ru/vacancy/36395822
обязанности:
https://spb.hh.ru/vacancy/35092323
обязанности:
https://spb.hh.ru/vacancy/36333894
обязанности
https://spb.hh.ru/vacancy/36239761
должностные обязанности:
https://spb.hh.ru/vacancy/36236040
обязанности:
https://spb.hh.ru/vacancy/36073532
https://spb.hh.ru/vacancy/33761446
обязанности:
https://spb.hh.ru/vacancy/36217892
обязанности:
https://spb.hh.ru/vacancy/36415284
обязанности:
https://spb.hh.ru/vacancy/35948527
обязанности:
https://spb.hh.ru/vacancy/36343760
обязанности:
https://spb.hh.ru/vacancy/36259289
обязанности:
https://spb.hh.ru/vacancy/36317090
обязанности:
https://spb.hh.ru/vacancy/36109005
обязанности:
https://spb.hh.ru/vacancy/35026204
https://spb.hh.ru/vacancy/36382582
https://spb.hh.ru/vacancy/36438098
обязанности:
https://spb.hh.ru/vacancy/33814876
обязанности:
https://spb.hh.ru/vacancy/36375362
обязанности:
https:/

обязанности:
https://spb.hh.ru/vacancy/36099699
https://spb.hh.ru/vacancy/36302008
обязанности:
https://spb.hh.ru/vacancy/34773491
обязанности
https://spb.hh.ru/vacancy/36280586
обязанности:
https://spb.hh.ru/vacancy/36215927
обязанности:
https://spb.hh.ru/vacancy/36233842
обязанности:
https://spb.hh.ru/vacancy/36456867
обязанности:
https://spb.hh.ru/vacancy/36526709
обязанности:
https://spb.hh.ru/vacancy/36170193
обязанности:
https://spb.hh.ru/vacancy/36151871
обязанности:
https://spb.hh.ru/vacancy/35812786
https://spb.hh.ru/vacancy/36369246
обязанности:
https://spb.hh.ru/vacancy/36302211
https://spb.hh.ru/vacancy/34828502
обязанности:
https://spb.hh.ru/vacancy/36064009
обязанности:
https://spb.hh.ru/vacancy/36331970
https://spb.hh.ru/vacancy/36146977
обязанности:
https://spb.hh.ru/vacancy/36323268
для данной работы вы должны обладать активной, бодрой, нацеленной на развитие жизненной позицией, получать удовольствие от своей работы, в которую будут входить основные должностные обязанн

обязанности:
https://spb.hh.ru/vacancy/36257550
обязанности
https://spb.hh.ru/vacancy/36115051
обязанности:
https://spb.hh.ru/vacancy/36444286
должностные обязанности:
https://spb.hh.ru/vacancy/36261559
обязанности:
https://spb.hh.ru/vacancy/36234817
обязанности:
https://spb.hh.ru/vacancy/35768183
обязанности
https://spb.hh.ru/vacancy/36174912
https://spb.hh.ru/vacancy/36258305
обязанности:
https://spb.hh.ru/vacancy/36426089
обязанности:
https://spb.hh.ru/vacancy/36319577
https://spb.hh.ru/vacancy/35646536
обязанности:
https://spb.hh.ru/vacancy/36161913
обязанности:
https://spb.hh.ru/vacancy/35877896
обязанности:
https://spb.hh.ru/vacancy/36128529


In [39]:
data  = get_vacancies(40, 'https://spb.hh.ru/catalog/Transport-Logistika')
data.to_csv(r'Z:\Collecting_data_openedu\18.csv')

обязанности:
https://spb.hh.ru/vacancy/36509966
https://spb.hh.ru/vacancy/36524757
https://spb.hh.ru/vacancy/36118137
https://spb.hh.ru/vacancy/36494572
https://spb.hh.ru/vacancy/30815513
обязанности:
https://spb.hh.ru/vacancy/35868099
https://spb.hh.ru/vacancy/36446388
обязанности:
https://spb.hh.ru/vacancy/35864814
https://spb.hh.ru/vacancy/35211517
https://spb.hh.ru/vacancy/27916453
https://spb.hh.ru/vacancy/33388007
https://spb.hh.ru/vacancy/27917074
обязанности: 
https://spb.hh.ru/vacancy/35229599
обязанности:
https://spb.hh.ru/vacancy/34304615
обязанности
https://spb.hh.ru/vacancy/36374308
https://spb.hh.ru/vacancy/35935645
обязанности:
https://spb.hh.ru/vacancy/36290591
https://spb.hh.ru/vacancy/36358572
https://spb.hh.ru/vacancy/36448544
обязанности:
https://spb.hh.ru/vacancy/36449651
обязанности:
https://spb.hh.ru/vacancy/36494726
обязанности:
https://spb.hh.ru/vacancy/36041401
обязанности:
https://spb.hh.ru/vacancy/36409053
обязанности:
https://spb.hh.ru/vacancy/36186821
обяз

https://spb.hh.ru/vacancy/36213082
https://spb.hh.ru/vacancy/36428227
обязанности:
https://spb.hh.ru/vacancy/36425882
обязанности:
https://spb.hh.ru/vacancy/36123119
https://spb.hh.ru/vacancy/35459354
обязанности:
https://spb.hh.ru/vacancy/35154059
обязанности:
https://spb.hh.ru/vacancy/33773562
обязанности:
https://spb.hh.ru/vacancy/36119095
обязанности:
https://spb.hh.ru/vacancy/35325325
https://spb.hh.ru/vacancy/36498295
в ваши обязанности будет входить:
https://spb.hh.ru/vacancy/34705304
https://spb.hh.ru/vacancy/36258127
https://spb.hh.ru/vacancy/36410382
обязанности:
https://spb.hh.ru/vacancy/36347669
https://spb.hh.ru/vacancy/36508449
https://spb.hh.ru/vacancy/36241099
обязанности:
https://spb.hh.ru/vacancy/36460779
обязанности:
https://spb.hh.ru/vacancy/36479542
обязанности менеджера по маркетингу:
https://spb.hh.ru/vacancy/36097951
обязанности:
https://spb.hh.ru/vacancy/36270515
обязанности
https://spb.hh.ru/vacancy/36176396
обязанности:
https://spb.hh.ru/vacancy/36350250
обяз

обязанности:
https://spb.hh.ru/vacancy/36435488
обязанности:
https://spb.hh.ru/vacancy/36443638
обязанности машиниста:
https://spb.hh.ru/vacancy/23254240
обязанности:
https://spb.hh.ru/vacancy/36254343
обязанности
https://spb.hh.ru/vacancy/36443385
в ваши обязанности будет входить:
https://spb.hh.ru/vacancy/35514696
обязанности:
https://spb.hh.ru/vacancy/36427913
обязанности:
https://spb.hh.ru/vacancy/36110136
обязанности:
https://spb.hh.ru/vacancy/36280228
обязанности:
https://spb.hh.ru/vacancy/36063889
обязанности:
https://spb.hh.ru/vacancy/36444631
https://spb.hh.ru/vacancy/36449540
https://spb.hh.ru/vacancy/36217081
обязанности:
https://spb.hh.ru/vacancy/36282634
обязанности:
https://spb.hh.ru/vacancy/35218562
обязанности:
https://spb.hh.ru/vacancy/36449121
https://spb.hh.ru/vacancy/36440301
обязанности:
https://spb.hh.ru/vacancy/36402184
обязанности:
https://spb.hh.ru/vacancy/34433323
https://spb.hh.ru/vacancy/36122601
обязанности:
https://spb.hh.ru/vacancy/35555018
https://spb.hh

https://spb.hh.ru/vacancy/35884200
обязанности:
https://spb.hh.ru/vacancy/36318383
https://spb.hh.ru/vacancy/35894301
обязанности
https://spb.hh.ru/vacancy/36305365
https://spb.hh.ru/vacancy/36298009
https://spb.hh.ru/vacancy/36313159
обязанности:
https://spb.hh.ru/vacancy/36307763
https://spb.hh.ru/vacancy/36301413
обязанности:
https://spb.hh.ru/vacancy/36294895
обязанности:
https://spb.hh.ru/vacancy/36105493
обязанности:
https://spb.hh.ru/vacancy/36105691
обязанности:
https://spb.hh.ru/vacancy/36395110
https://spb.hh.ru/vacancy/35495035
обязанности:
https://spb.hh.ru/vacancy/36065871
обязанности:
https://spb.hh.ru/vacancy/36293333
обязанности:
https://spb.hh.ru/vacancy/36285019
обязанности:
https://spb.hh.ru/vacancy/36285044
обязанности:
https://spb.hh.ru/vacancy/35841836
https://spb.hh.ru/vacancy/36335099
https://spb.hh.ru/vacancy/36286258
обязанности:
https://spb.hh.ru/vacancy/36278603
обязанности:
https://spb.hh.ru/vacancy/35834737
обязанности:
https://spb.hh.ru/vacancy/34680201
о

https://spb.hh.ru/vacancy/36501730
обязанности:
https://spb.hh.ru/vacancy/36433244
основные обязанности
https://spb.hh.ru/vacancy/36195956
обязанности:
https://spb.hh.ru/vacancy/36290178
обязанности:
https://spb.hh.ru/vacancy/36319274
обязанности:
https://spb.hh.ru/vacancy/36437524
обязанности:
https://spb.hh.ru/vacancy/36407848
https://spb.hh.ru/vacancy/36432362
обязанности:
https://spb.hh.ru/vacancy/36442703
https://spb.hh.ru/vacancy/36338484
обязанности:
https://spb.hh.ru/vacancy/36510235
обязанности:
https://spb.hh.ru/vacancy/36521985
https://spb.hh.ru/vacancy/36425702
https://spb.hh.ru/vacancy/36410200
обязанности:
https://spb.hh.ru/vacancy/33805616
обязанности:
https://spb.hh.ru/vacancy/36515022
https://spb.hh.ru/vacancy/36455874
обязанности:
https://spb.hh.ru/vacancy/36505538
обязанности:
https://spb.hh.ru/vacancy/36206576
https://spb.hh.ru/vacancy/36317600
https://spb.hh.ru/vacancy/36111441
https://spb.hh.ru/vacancy/36363807
обязанности:
https://spb.hh.ru/vacancy/36456624
обяза

...

обязанности:
https://spb.hh.ru/vacancy/36244383
обязанности: 
https://spb.hh.ru/vacancy/36242980
обязанности:
https://spb.hh.ru/vacancy/36356073
https://spb.hh.ru/vacancy/36213185
обязанности:
https://spb.hh.ru/vacancy/35895018
обязанности:
https://spb.hh.ru/vacancy/36350221
https://spb.hh.ru/vacancy/36202031
обязанности:
https://spb.hh.ru/vacancy/36225739
обязанности:
https://spb.hh.ru/vacancy/36380871
обязанности:
https://spb.hh.ru/vacancy/36516569
должностные обязанности:
https://spb.hh.ru/vacancy/32791125
обязанности:
https://spb.hh.ru/vacancy/36449078
обязанности:
https://spb.hh.ru/vacancy/36327348
обязанности:
https://spb.hh.ru/vacancy/36123168
https://spb.hh.ru/vacancy/36298952
обязанности:
https://spb.hh.ru/vacancy/36288620
обязанности:
https://spb.hh.ru/vacancy/35183678
обязанности:
https://spb.hh.ru/vacancy/36504263
обязанности:
https://spb.hh.ru/vacancy/36359694
обязанности:
https://spb.hh.ru/vacancy/35940743
обязанности:
https://spb.hh.ru/vacancy/36407063
обязанности:
https

обязанности:
https://spb.hh.ru/vacancy/36261995
обязанности:
https://spb.hh.ru/vacancy/36223032
обязанности:
https://spb.hh.ru/vacancy/35957250
https://spb.hh.ru/vacancy/36281439
обязанности:
https://spb.hh.ru/vacancy/36087593
https://spb.hh.ru/vacancy/35800098
обязанности:
https://spb.hh.ru/vacancy/35411515
обязанности:
https://spb.hh.ru/vacancy/36423501
https://spb.hh.ru/vacancy/35531159
https://spb.hh.ru/vacancy/36094804
обязанности:
https://spb.hh.ru/vacancy/36370492
обязанности:
https://spb.hh.ru/vacancy/36236546
обязанности:
https://spb.hh.ru/vacancy/36088067
обязанности:
https://spb.hh.ru/vacancy/32663868
обязанности:
https://spb.hh.ru/vacancy/36400121
обязанности:
https://spb.hh.ru/vacancy/36409208
https://spb.hh.ru/vacancy/36111131
обязанности:
https://spb.hh.ru/vacancy/36393351
обязанности:
https://spb.hh.ru/vacancy/36522004
должностные обязанности
https://spb.hh.ru/vacancy/36000487
обязанности:
https://spb.hh.ru/vacancy/36293118
обязанности:
https://spb.hh.ru/vacancy/3624381

обязанности:
https://spb.hh.ru/vacancy/36243006
обязанности
https://spb.hh.ru/vacancy/36456911
обязанности:
https://spb.hh.ru/vacancy/36389635
обязанности:
https://spb.hh.ru/vacancy/36281438
https://spb.hh.ru/vacancy/36136298
https://spb.hh.ru/vacancy/36060818
должностные обязанности:
https://spb.hh.ru/vacancy/36156818
обязанности:
https://spb.hh.ru/vacancy/36516213
обязанности:
https://spb.hh.ru/vacancy/35677201
https://spb.hh.ru/vacancy/36223837
обязанности:
https://spb.hh.ru/vacancy/36058812
должностные обязанности:
https://spb.hh.ru/vacancy/35723132
обязанности:
https://spb.hh.ru/vacancy/36414478
обязанности:
https://spb.hh.ru/vacancy/36199315
https://spb.hh.ru/vacancy/36501644
обязанности:
https://spb.hh.ru/vacancy/34390211
обязанности:
https://spb.hh.ru/vacancy/36165683
обязанности:
https://spb.hh.ru/vacancy/36068660
обязанности:
https://spb.hh.ru/vacancy/36211050
https://spb.hh.ru/vacancy/35204310
обязанности:
https://spb.hh.ru/vacancy/33250600
обязанности
https://spb.hh.ru/vaca

https://spb.hh.ru/vacancy/36527657
https://spb.hh.ru/vacancy/36526032
обязанности:
https://spb.hh.ru/vacancy/36464798
обязанности:
https://spb.hh.ru/vacancy/36107876
обязанности:
https://spb.hh.ru/vacancy/35309241
https://spb.hh.ru/vacancy/36469768
должностные обязанности:
https://spb.hh.ru/vacancy/36423069
обязанности:
https://spb.hh.ru/vacancy/36098742
https://spb.hh.ru/vacancy/36328353
https://spb.hh.ru/vacancy/36527995
обязанности:
https://spb.hh.ru/vacancy/35043347
обязанности:
https://spb.hh.ru/vacancy/36067552
обязанности:
https://spb.hh.ru/vacancy/36113545
обязанности:
https://spb.hh.ru/vacancy/35874255
обязанности: 
https://spb.hh.ru/vacancy/36092513
обязанности:
https://spb.hh.ru/vacancy/36164351
обязанности:
https://spb.hh.ru/vacancy/35945481
обязанности:
https://spb.hh.ru/vacancy/36283216
https://spb.hh.ru/vacancy/36499602
должностные обязанности:
https://spb.hh.ru/vacancy/36465022
https://spb.hh.ru/vacancy/35802644
обязанности:
https://spb.hh.ru/vacancy/36334778
обязанност

https://spb.hh.ru/vacancy/36526052
должностные обязанности:
https://spb.hh.ru/vacancy/36264960
https://spb.hh.ru/vacancy/36287949


In [40]:
data  = get_vacancies(39, 'https://spb.hh.ru/catalog/Turizm-Gostinicy-Restorany')
data.to_csv(r'Z:\Collecting_data_openedu\19.csv')

https://spb.hh.ru/vacancy/35942415
обязанности:
https://spb.hh.ru/vacancy/36217468
обязанности:
https://spb.hh.ru/vacancy/34304615
https://spb.hh.ru/vacancy/36186393
в ваши обязанности будет входить:
https://spb.hh.ru/vacancy/36378445
https://spb.hh.ru/vacancy/36367136
обязанности:
https://spb.hh.ru/vacancy/32381738
https://spb.hh.ru/vacancy/36178271
обязанности:
https://spb.hh.ru/vacancy/35997359
обязанности:
https://spb.hh.ru/vacancy/32381742
обязанности:
https://spb.hh.ru/vacancy/35362906
https://spb.hh.ru/vacancy/35283389
обязанности:
https://spb.hh.ru/vacancy/36211877
обязанности:
https://spb.hh.ru/vacancy/36397247
https://spb.hh.ru/vacancy/35398330
обязанности:
https://spb.hh.ru/vacancy/36279384
обязанности: 
https://spb.hh.ru/vacancy/36217408
обязанности:
https://spb.hh.ru/vacancy/36275354
обязанности:
https://spb.hh.ru/vacancy/35362905
https://spb.hh.ru/vacancy/36074991
обязанности:
https://spb.hh.ru/vacancy/35725795
обязанности:
https://spb.hh.ru/vacancy/35362904
обязанности:


https://spb.hh.ru/vacancy/36462897
https://spb.hh.ru/vacancy/36426442
обязанности:
https://spb.hh.ru/vacancy/36186142
https://spb.hh.ru/vacancy/34098830
https://spb.hh.ru/vacancy/36177758
обязанности:
https://spb.hh.ru/vacancy/36216118
обязанности:
https://spb.hh.ru/vacancy/36449156
обязанности:
https://spb.hh.ru/vacancy/36176004
должностные обязанности:
https://spb.hh.ru/vacancy/31650066
обязанности:
https://spb.hh.ru/vacancy/36204883
https://spb.hh.ru/vacancy/36186703
обязанности:
https://spb.hh.ru/vacancy/36430415
https://spb.hh.ru/vacancy/36407956
обязанности:
https://spb.hh.ru/vacancy/36180377
https://spb.hh.ru/vacancy/34103560
https://spb.hh.ru/vacancy/36245031
обязанности:
https://spb.hh.ru/vacancy/36179417
https://spb.hh.ru/vacancy/36445786
обязанности:
https://spb.hh.ru/vacancy/36316388
обязанности:
https://spb.hh.ru/vacancy/35415766
https://spb.hh.ru/vacancy/36431469
https://spb.hh.ru/vacancy/36447160
обязанности:
https://spb.hh.ru/vacancy/36449404
https://spb.hh.ru/vacancy/3

обязанности:
https://spb.hh.ru/vacancy/36280697
обязанности:
https://spb.hh.ru/vacancy/36320111
https://spb.hh.ru/vacancy/35045660
https://spb.hh.ru/vacancy/35645487
обязанности:
https://spb.hh.ru/vacancy/36067315
https://spb.hh.ru/vacancy/34680356
https://spb.hh.ru/vacancy/36395872
обязанности:
https://spb.hh.ru/vacancy/36318004
https://spb.hh.ru/vacancy/35628703
https://spb.hh.ru/vacancy/35198283
https://spb.hh.ru/vacancy/34245903
https://spb.hh.ru/vacancy/35784626
https://spb.hh.ru/vacancy/36300436
обязанности
https://spb.hh.ru/vacancy/36296727
обязанности:
https://spb.hh.ru/vacancy/36243702
обязанности:
https://spb.hh.ru/vacancy/36263404
обязанности:
https://spb.hh.ru/vacancy/36279076
обязанности:
https://spb.hh.ru/vacancy/35360758
обязанности:
https://spb.hh.ru/vacancy/36186472
https://spb.hh.ru/vacancy/36084147
обязанности:
https://spb.hh.ru/vacancy/36191982
обязанности:
https://spb.hh.ru/vacancy/34869171
https://spb.hh.ru/vacancy/36019910
https://spb.hh.ru/vacancy/36289544
обяза

обязанности:
https://spb.hh.ru/vacancy/35684291
https://spb.hh.ru/vacancy/36063914
обязанности:
https://spb.hh.ru/vacancy/36525185
обязанности:
https://spb.hh.ru/vacancy/36452564
обязанности:
https://spb.hh.ru/vacancy/36454449
обязанности:
https://spb.hh.ru/vacancy/36507812
https://spb.hh.ru/vacancy/36314467
обязанности:
https://spb.hh.ru/vacancy/36180424
обязанности:
https://spb.hh.ru/vacancy/36405057
обязанности:
https://spb.hh.ru/vacancy/36323943
обязанности:
https://spb.hh.ru/vacancy/36206372
обязанности:
https://spb.hh.ru/vacancy/36413627
обязанности:
https://spb.hh.ru/vacancy/36236860
обязанности:
https://spb.hh.ru/vacancy/36432906
обязанности:
https://spb.hh.ru/vacancy/36448656
обязанности:
https://spb.hh.ru/vacancy/36097998
обязанности:
https://spb.hh.ru/vacancy/36287055
обязанности:
https://spb.hh.ru/vacancy/36109497
обязанности:
https://spb.hh.ru/vacancy/36465526
обязанности:
https://spb.hh.ru/vacancy/36470663
обязанности:
https://spb.hh.ru/vacancy/36294437
https://spb.hh.ru/

обязанности:
https://spb.hh.ru/vacancy/36065221
обязанности:
https://spb.hh.ru/vacancy/35535429
обязанности:
https://spb.hh.ru/vacancy/36308602
обязанности:
https://spb.hh.ru/vacancy/36297612
https://spb.hh.ru/vacancy/35684295
обязанности:
https://spb.hh.ru/vacancy/33102784
обязанности:
https://spb.hh.ru/vacancy/36318006
https://spb.hh.ru/vacancy/35504053
обязанности:
https://spb.hh.ru/vacancy/35611354
обязанности:
https://spb.hh.ru/vacancy/36281384
обязанности
https://spb.hh.ru/vacancy/36121083
https://spb.hh.ru/vacancy/36267735
https://spb.hh.ru/vacancy/35969197
обязанности:
https://spb.hh.ru/vacancy/36084711
ваши обязанности:
https://spb.hh.ru/vacancy/36521624
обязанности:
https://spb.hh.ru/vacancy/36212604
https://spb.hh.ru/vacancy/36106956
обязанности:
https://spb.hh.ru/vacancy/36460320
обязанности:
https://spb.hh.ru/vacancy/36266918
https://spb.hh.ru/vacancy/36382383
https://spb.hh.ru/vacancy/36107898
обязанности:
https://spb.hh.ru/vacancy/36043302
https://spb.hh.ru/vacancy/35098

обязанности:
https://spb.hh.ru/vacancy/36082981
должностные обязанности:
https://spb.hh.ru/vacancy/36229383
https://spb.hh.ru/vacancy/36068858
обязанности:
https://spb.hh.ru/vacancy/35905931
https://spb.hh.ru/vacancy/35802644
https://spb.hh.ru/vacancy/31976545
обязанности:
https://spb.hh.ru/vacancy/36254962
https://spb.hh.ru/vacancy/36331659
https://spb.hh.ru/vacancy/36124110
https://spb.hh.ru/vacancy/35673636
https://spb.hh.ru/vacancy/36070110
обязанности:
https://spb.hh.ru/vacancy/36079122
обязанности:
https://spb.hh.ru/vacancy/36288010
https://spb.hh.ru/vacancy/36059110
обязанности:
https://spb.hh.ru/vacancy/35066164
https://spb.hh.ru/vacancy/35218086
обязанности:
https://spb.hh.ru/vacancy/34665498
обязанности:
https://spb.hh.ru/vacancy/36187234
обязанности:
https://spb.hh.ru/vacancy/36111533
https://spb.hh.ru/vacancy/35646891
обязанности:
https://spb.hh.ru/vacancy/36269996
https://spb.hh.ru/vacancy/35937087
https://spb.hh.ru/vacancy/36199819
https://spb.hh.ru/vacancy/31333874
обяза

https://spb.hh.ru/vacancy/35683934
https://spb.hh.ru/vacancy/36459448
https://spb.hh.ru/vacancy/36358192
https://spb.hh.ru/vacancy/36529056
https://spb.hh.ru/vacancy/35146407
https://spb.hh.ru/vacancy/36253357
обязанности:
https://spb.hh.ru/vacancy/30673593
https://spb.hh.ru/vacancy/35146413
https://spb.hh.ru/vacancy/36273756
должностные обязанности:
https://spb.hh.ru/vacancy/36386851
https://spb.hh.ru/vacancy/36130671
https://spb.hh.ru/vacancy/35894431
обязанности:
https://spb.hh.ru/vacancy/36146056
https://spb.hh.ru/vacancy/36013464
https://spb.hh.ru/vacancy/36334466
https://spb.hh.ru/vacancy/35778504
https://spb.hh.ru/vacancy/36244354
обязанности:
https://spb.hh.ru/vacancy/36142945
https://spb.hh.ru/vacancy/36392400
обязанности:
https://spb.hh.ru/vacancy/35765787
https://spb.hh.ru/vacancy/34754725
https://spb.hh.ru/vacancy/35151501
обязанности:
https://spb.hh.ru/vacancy/36146040
https://spb.hh.ru/vacancy/35045441
https://spb.hh.ru/vacancy/35146395
https://spb.hh.ru/vacancy/36529053


обязанности:
https://spb.hh.ru/vacancy/36135988
обязанности:
https://spb.hh.ru/vacancy/36140300
https://spb.hh.ru/vacancy/36145624
обязанности:
https://spb.hh.ru/vacancy/36167461
https://spb.hh.ru/vacancy/36169466
обязанности:
https://spb.hh.ru/vacancy/36169826
обязанности:
https://spb.hh.ru/vacancy/36172403
обязанности:
https://spb.hh.ru/vacancy/36178324
обязанности:
https://spb.hh.ru/vacancy/36178849
https://spb.hh.ru/vacancy/36209520
https://spb.hh.ru/vacancy/36228719
https://spb.hh.ru/vacancy/36233422
обязанности: 
https://spb.hh.ru/vacancy/36233525
https://spb.hh.ru/vacancy/36256929
https://spb.hh.ru/vacancy/36257246
https://spb.hh.ru/vacancy/36259736
https://spb.hh.ru/vacancy/36260562
https://spb.hh.ru/vacancy/36262063
обязанности:
https://spb.hh.ru/vacancy/36262408
обязанности:
https://spb.hh.ru/vacancy/36271789
https://spb.hh.ru/vacancy/36285157
обязанности:
https://spb.hh.ru/vacancy/36287885
обязанности:
https://spb.hh.ru/vacancy/36289453
обязанности:
https://spb.hh.ru/vacancy

обязанности:
https://spb.hh.ru/vacancy/36093545
обязанности:
https://spb.hh.ru/vacancy/35296369
обязанности:
https://spb.hh.ru/vacancy/36072452
https://spb.hh.ru/vacancy/36156302
https://spb.hh.ru/vacancy/36186771
https://spb.hh.ru/vacancy/36135384
обязанности:
https://spb.hh.ru/vacancy/35637794
https://spb.hh.ru/vacancy/36159064
обязанности:
https://spb.hh.ru/vacancy/36170309
https://spb.hh.ru/vacancy/36082019
должностные обязанности:
https://spb.hh.ru/vacancy/36140162
обязанности:
https://spb.hh.ru/vacancy/36108917
https://spb.hh.ru/vacancy/36113991
обязанности:
https://spb.hh.ru/vacancy/36117674
обязанности:
https://spb.hh.ru/vacancy/36117742
https://spb.hh.ru/vacancy/36131654
https://spb.hh.ru/vacancy/36141235
обязанности:
https://spb.hh.ru/vacancy/36145299
обязанности:
https://spb.hh.ru/vacancy/36063998
https://spb.hh.ru/vacancy/36059282
обязанности:
https://spb.hh.ru/vacancy/36123093
https://spb.hh.ru/vacancy/36070240
обязанности:
https://spb.hh.ru/vacancy/36095797
обязанности:
h

In [41]:
data  = get_vacancies(23, 'https://spb.hh.ru/catalog/Upravlenie-personalom-Treningi')
data.to_csv(r'Z:\Collecting_data_openedu\20.csv')

https://spb.hh.ru/vacancy/36191145
обязанности:
https://spb.hh.ru/vacancy/36321295
обязанности:
https://spb.hh.ru/vacancy/36397186
обязанности:
https://spb.hh.ru/vacancy/36131166
обязанности:
https://spb.hh.ru/vacancy/36441401
обязанности
https://spb.hh.ru/vacancy/36317933
https://spb.hh.ru/vacancy/35885602
https://spb.hh.ru/vacancy/36186393
https://spb.hh.ru/vacancy/36204533
обязанности:
https://spb.hh.ru/vacancy/36364415
обязанности:
https://spb.hh.ru/vacancy/36290599
обязанности:
https://spb.hh.ru/vacancy/35772033
https://spb.hh.ru/vacancy/35885292
обязанности:
https://spb.hh.ru/vacancy/36444220
https://spb.hh.ru/vacancy/36378632
обязанности:
https://spb.hh.ru/vacancy/36360297
https://spb.hh.ru/vacancy/36317019
https://spb.hh.ru/vacancy/35713406
https://spb.hh.ru/vacancy/36315146
обязанности:
https://spb.hh.ru/vacancy/34980999
https://spb.hh.ru/vacancy/35814965
https://spb.hh.ru/vacancy/35656806
https://spb.hh.ru/vacancy/36521477
https://spb.hh.ru/vacancy/36191972
https://spb.hh.ru/

https://spb.hh.ru/vacancy/35281701
https://spb.hh.ru/vacancy/36486738
https://spb.hh.ru/vacancy/34648876
обязанности:
https://spb.hh.ru/vacancy/35805131
обязанности:
https://spb.hh.ru/vacancy/36025024
в ваши обязанности будет входить:
https://spb.hh.ru/vacancy/36189695
https://spb.hh.ru/vacancy/36092475
обязанности:
https://spb.hh.ru/vacancy/34047487
https://spb.hh.ru/vacancy/36121771
https://spb.hh.ru/vacancy/36061095
обязанности:
https://spb.hh.ru/vacancy/36441271
обязанности:
https://spb.hh.ru/vacancy/35891083
обязанности:
https://spb.hh.ru/vacancy/36456020
https://spb.hh.ru/vacancy/36430704
в ваши обязанности будут входить:
https://spb.hh.ru/vacancy/34020529
https://spb.hh.ru/vacancy/36329434
обязанности:
https://spb.hh.ru/vacancy/36196825
обязанности:
https://spb.hh.ru/vacancy/35464427
https://spb.hh.ru/vacancy/36329680
https://spb.hh.ru/vacancy/36334760
https://spb.hh.ru/vacancy/34507284
обязанности:
https://spb.hh.ru/vacancy/36437847
обязанности:
https://spb.hh.ru/vacancy/363827

https://spb.hh.ru/vacancy/36390337
обязанности:
https://spb.hh.ru/vacancy/35428854
обязанности:
https://spb.hh.ru/vacancy/36375261
обязанности:
https://spb.hh.ru/vacancy/36109876
обязанности:
https://spb.hh.ru/vacancy/36084312
https://spb.hh.ru/vacancy/36377677
https://spb.hh.ru/vacancy/35743738
обязанности:
https://spb.hh.ru/vacancy/34390211
https://spb.hh.ru/vacancy/35949685
обязанности:
https://spb.hh.ru/vacancy/35505305
https://spb.hh.ru/vacancy/35950502
обязанности:
https://spb.hh.ru/vacancy/35875453
обязанности:
https://spb.hh.ru/vacancy/36215622
https://spb.hh.ru/vacancy/35716621
обязанности:
https://spb.hh.ru/vacancy/36318787
https://spb.hh.ru/vacancy/33682125
обязанности:
https://spb.hh.ru/vacancy/35784804
https://spb.hh.ru/vacancy/35997530
https://spb.hh.ru/vacancy/36351579
https://spb.hh.ru/vacancy/36054027
https://spb.hh.ru/vacancy/36070364
обязанности:
https://spb.hh.ru/vacancy/36413994
https://spb.hh.ru/vacancy/36375550
https://spb.hh.ru/vacancy/36408706
обязанности:
http

https://spb.hh.ru/vacancy/36108019
https://spb.hh.ru/vacancy/36114046
обязанности:
https://spb.hh.ru/vacancy/35810816
профессиональные обязанности:
https://spb.hh.ru/vacancy/35220676
обязанности:
https://spb.hh.ru/vacancy/36191740
https://spb.hh.ru/vacancy/36100200
обязанности:
https://spb.hh.ru/vacancy/36081625
обязанности управляющего
https://spb.hh.ru/vacancy/36108593
https://spb.hh.ru/vacancy/36081264
обязанности:
https://spb.hh.ru/vacancy/36228161
https://spb.hh.ru/vacancy/36088035
обязанности:
https://spb.hh.ru/vacancy/35645976
https://spb.hh.ru/vacancy/35630692
https://spb.hh.ru/vacancy/36100885
обязанности:
https://spb.hh.ru/vacancy/36257037
https://spb.hh.ru/vacancy/36061103
обязанности:
https://spb.hh.ru/vacancy/36168214
обязанности:
https://spb.hh.ru/vacancy/36138505
обязанности:
https://spb.hh.ru/vacancy/36063428
обязанности:
https://spb.hh.ru/vacancy/36132001
https://spb.hh.ru/vacancy/36068959
обязанности:
https://spb.hh.ru/vacancy/36056111
https://spb.hh.ru/vacancy/360656

обязанности:
https://spb.hh.ru/vacancy/36339790
обязанности:
https://spb.hh.ru/vacancy/36118981
обязанности:
https://spb.hh.ru/vacancy/36280211
обязанности:
https://spb.hh.ru/vacancy/36163012
https://spb.hh.ru/vacancy/35310939
обязанности:
https://spb.hh.ru/vacancy/36366511
обязанности:
https://spb.hh.ru/vacancy/36372987
https://spb.hh.ru/vacancy/36310773
обязанности:
https://spb.hh.ru/vacancy/36218062
https://spb.hh.ru/vacancy/36161653
https://spb.hh.ru/vacancy/36216984
обязанности:
https://spb.hh.ru/vacancy/36412788
https://spb.hh.ru/vacancy/36101986
https://spb.hh.ru/vacancy/35711514
обязанности:
https://spb.hh.ru/vacancy/36409711
обязанности:
https://spb.hh.ru/vacancy/36015892
обязанности:
https://spb.hh.ru/vacancy/36122707
обязанности:
https://spb.hh.ru/vacancy/36339222
обязанности:
https://spb.hh.ru/vacancy/36401134
https://spb.hh.ru/vacancy/36118078
обязанности:
https://spb.hh.ru/vacancy/35146623
профессиональные обязанности:
https://spb.hh.ru/vacancy/36109420
https://spb.hh.ru/

обязанности:
https://spb.hh.ru/vacancy/36001867
обязанности:
https://spb.hh.ru/vacancy/35553923
https://spb.hh.ru/vacancy/35967227
https://spb.hh.ru/vacancy/34245903
https://spb.hh.ru/vacancy/29935737
https://spb.hh.ru/vacancy/36388898
https://spb.hh.ru/vacancy/35966362
обязанности:
https://spb.hh.ru/vacancy/32629569
https://spb.hh.ru/vacancy/36275974
https://spb.hh.ru/vacancy/35967244
обязанности:
https://spb.hh.ru/vacancy/34789136
обязанности:
https://spb.hh.ru/vacancy/35492999
обязанности:
https://spb.hh.ru/vacancy/35815452
https://spb.hh.ru/vacancy/36308885
обязанности: 
https://spb.hh.ru/vacancy/36114648
обязанности:
https://spb.hh.ru/vacancy/36216092
https://spb.hh.ru/vacancy/36287289
обязанности
https://spb.hh.ru/vacancy/34945999
https://spb.hh.ru/vacancy/35677683
обязанности:
https://spb.hh.ru/vacancy/35709320
обязанности:
https://spb.hh.ru/vacancy/36073856
обязанности
https://spb.hh.ru/vacancy/36087385
обязанности:
https://spb.hh.ru/vacancy/36134066
https://spb.hh.ru/vacancy/3

In [42]:
data  = get_vacancies(13, 'https://spb.hh.ru/catalog/Juristy')
data.to_csv(r'Z:\Collecting_data_openedu\21.csv')

https://spb.hh.ru/vacancy/36367136
обязанности:
https://spb.hh.ru/vacancy/34352709
https://spb.hh.ru/vacancy/36210758
https://spb.hh.ru/vacancy/36024649
https://spb.hh.ru/vacancy/36314852
обязанности
https://spb.hh.ru/vacancy/36073624
обязанности
https://spb.hh.ru/vacancy/35830805
обязанности:
https://spb.hh.ru/vacancy/35851952
обязанности:
https://spb.hh.ru/vacancy/36191478
в ваши обязанности будет входить
https://spb.hh.ru/vacancy/36316853
обязанности:
https://spb.hh.ru/vacancy/36286731
https://spb.hh.ru/vacancy/36319213
https://spb.hh.ru/vacancy/36369136
обязанности:
https://spb.hh.ru/vacancy/36213555
https://spb.hh.ru/vacancy/36495455
обязанности
https://spb.hh.ru/vacancy/36395839
https://spb.hh.ru/vacancy/29875926
обязанности:
https://spb.hh.ru/vacancy/36371758
https://spb.hh.ru/vacancy/27468544
обязанности:
https://spb.hh.ru/vacancy/35675777
обязанности:
https://spb.hh.ru/vacancy/36284035
https://spb.hh.ru/vacancy/27468471
основные обязанности:
https://spb.hh.ru/vacancy/36282411


обязанности:
https://spb.hh.ru/vacancy/36362023
профессиональные обязанности:
https://spb.hh.ru/vacancy/36447083
обязанности:
https://spb.hh.ru/vacancy/36444136
обязанности:
https://spb.hh.ru/vacancy/36335963
обязанности:
https://spb.hh.ru/vacancy/36319704
https://spb.hh.ru/vacancy/36459214
обязанности:
https://spb.hh.ru/vacancy/36278319
функциональные обязанности:
https://spb.hh.ru/vacancy/36382779
обязанности:
https://spb.hh.ru/vacancy/36464508
должностные обязанности:
https://spb.hh.ru/vacancy/36464771
обязанности:
https://spb.hh.ru/vacancy/36359515
https://spb.hh.ru/vacancy/36383047
https://spb.hh.ru/vacancy/36235667
обязанности:
https://spb.hh.ru/vacancy/36088126
обязанности:
https://spb.hh.ru/vacancy/36405407
обязанности:
https://spb.hh.ru/vacancy/36233769
обязанности:
https://spb.hh.ru/vacancy/36376849
обязанности:
https://spb.hh.ru/vacancy/36410602
обязанности:
https://spb.hh.ru/vacancy/36313556
https://spb.hh.ru/vacancy/36406121
обязанности:
https://spb.hh.ru/vacancy/36373391


https://spb.hh.ru/vacancy/36340951
обязанности:
https://spb.hh.ru/vacancy/36341391
обязанности:
https://spb.hh.ru/vacancy/36124109
https://spb.hh.ru/vacancy/35279373
https://spb.hh.ru/vacancy/36325580
обязанности:
https://spb.hh.ru/vacancy/36280502
обязанности:
https://spb.hh.ru/vacancy/36161913
https://spb.hh.ru/vacancy/33163986
обязанности:
https://spb.hh.ru/vacancy/36300732
обязанности:
https://spb.hh.ru/vacancy/36242104
https://spb.hh.ru/vacancy/35781685
обязанности
https://spb.hh.ru/vacancy/35852870
https://spb.hh.ru/vacancy/34923329
обязанности:
https://spb.hh.ru/vacancy/36066358
обязанности:
https://spb.hh.ru/vacancy/36393930
обязанности:
https://spb.hh.ru/vacancy/36303605
обязанности:
https://spb.hh.ru/vacancy/35960032
обязанности:
https://spb.hh.ru/vacancy/35883074
https://spb.hh.ru/vacancy/36143448
обязанности:
https://spb.hh.ru/vacancy/36385523
обязанности:
https://spb.hh.ru/vacancy/36097987
обязанности
https://spb.hh.ru/vacancy/35757282
обязанности:
https://spb.hh.ru/vacanc

обязанности:
https://spb.hh.ru/vacancy/36301612
обязанности:
https://spb.hh.ru/vacancy/36332688
обязанности: 
https://spb.hh.ru/vacancy/36246800
обязанности:
https://spb.hh.ru/vacancy/35466417
обязанности:
https://spb.hh.ru/vacancy/35868946
обязанности
https://spb.hh.ru/vacancy/36071769
обязанности:
https://spb.hh.ru/vacancy/36142706
обязанности:
https://spb.hh.ru/vacancy/36155025
обязанности
https://spb.hh.ru/vacancy/36167225
обязанности:
https://spb.hh.ru/vacancy/36238138
обязанности:
https://spb.hh.ru/vacancy/36261905
https://spb.hh.ru/vacancy/36080604
обязанности:
https://spb.hh.ru/vacancy/36082090
основные обязанности:
https://spb.hh.ru/vacancy/36265707
https://spb.hh.ru/vacancy/36085420
обязанности:
https://spb.hh.ru/vacancy/36124983
должностные обязанности:
https://spb.hh.ru/vacancy/36161825
обязанности:
https://spb.hh.ru/vacancy/36067371
обязанности:
https://spb.hh.ru/vacancy/36148032
обязанности:
https://spb.hh.ru/vacancy/36087371
обязанности:
https://spb.hh.ru/vacancy/3441554